# Introduction

This notebook presents an informal introduction to geometric algebra in Dyalog APL. It covers from the basic properties of vector spaces to the usage of projective geometric algebra for the solution of geometric problems. 

Distinctions are not made between theoretical algebra concepts and its implementation in APL.

#### Disclaimer

Some knowledge of linear algebra and vector spaces could be handy, but all concepts will be introduced. However, this is not a formal introduction to geometric algebra. The definitions are not rigorous, properties are used before being defined, and it is very far from being exhaustive in the treated topics. Some links are provided, but the linked content might differ from the content presented here. Please, refer to a proper book for a deeper understanding of geometric (or Clifford) algebras. Other introductions and additional resources can also be found online. Some suggestions:

- [David Hestenes web site](https://geocalc.clas.asu.edu/)
- [Rigid geometric algebra wiki](https://rigidgeometricalgebra.org/)
- [biVector.net](https://bivector.net/)
- [Alan Macdonald books](http://www.faculty.luther.edu/~macdonal/#geometric-algebra)

The code fragments are written in [Dyalog APL](https://www.dyalog.com/dyalog/dyalog-versions/182.htm) and they are an essential part of this document, including the comments. Therefore, being able to read APL is necessary. Nevertheless, all functions and operators are explained with enough detail to (hopefully) be understood with only a basic knowledge of the language.

## Preliminaries

In [1]:
)clear
⎕IO ← 0        ⍝ important
⎕PP ← 4        ⍝ just for display
⎕CT ← 1e¯10    ⍝ avoid rounding issues
'H'⎕CY'dfns'   ⍝ dfns' quaternions for comparison
Assert ← {⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}   ⍝ Assert by Roger Hui
Table  ← ≡{1<d←|⍺⍺ ⍵: ((d-1)⍨)∇∇(⍵⍵¨) ⍵ ⋄ ⍵⍵ ⍵}⍕¨(,[¯0.5])   ⍝ display as table

clear ws

# Vector spaces

A [vector space](https://en.wikipedia.org/wiki/Vector_space) is a set of elements which follow certain properties.

## Vector space properties

The `VS` function will tell us if the three values given as right argument, called *[vectors](https://en.wikipedia.org/wiki/Vector_(mathematics_and_physics))*, together with the two values given as left argument, called *[scalars](https://en.wikipedia.org/wiki/Scalar_(mathematics))*, staisfy the properties of vector spaces:

In [2]:
]dinput
VS ← {                                         ⍝ (V)ector (S)pace
    _ ← Assert (⊂⍬) ≡∘⍴¨ a b ← ⍺               ⍝ a and b are scalars
    _ ← Assert (⊂,1) ≡∘⍴∘⍴¨ u v w ← ⍵          ⍝ u, v and w are vectors (rank 1 arrays)
    _ ← Assert 2=/≢¨ u v w                     ⍝ u, v and w have the same length
                                               ⍝ properties:
    _ ← Assert v ≡ v + z ← ≠⍨ v                ⍝   neutral element for addition (0)
    _ ← Assert v ≡ v × o ← =⍨ v                ⍝   neutral element for multiplication (1)
    _ ← Assert z ≡ v + -v                      ⍝   inverse element for addition
    _ ← Assert (v - w) ≡ v + -w                ⍝   substraction function
    _ ← Assert (v + w) ≡ w + v                 ⍝   commutative addition
    _ ← Assert (a × v) ≡ v × a                 ⍝   commutative product with scalars
    _ ← Assert (u + v + w) ≡ (u + v) + w       ⍝   associative addition
    _ ← Assert (v × a × b) ≡ (v × a) × b       ⍝   associative product with scalars
    _ ← Assert (a × v + w) ≡ (a × v) + a × w   ⍝   distributive over addition of vectors
    _ ← Assert (v × a + b) ≡ (v × a) + v × b   ⍝   distributive over addition of scalars
    1
}

Let's generate some sample data (two scalars and three 3D vectors) and check that they fulfill these properties:

In [3]:
'ab'  Table a b   ←  5×?  2⍴0   ⍝ 2 random scalars
'uvw' Table u v w ← ↓5×?3 3⍴0   ⍝ 3 random 3D vectors
a b VS u v w                    ⍝ vector space?

┌─────┬─────┐
│a │b │
├─────┼─────┤
│4.652│4.971│
└─────┴─────┘

┌───────────────────┬────────────────┬───────────────────┐
│u │v │w │
├───────────────────┼────────────────┼───────────────────┤
│0.01894 2.846 2.178│3.701 1.049 3.84│0.02235 1.546 4.125│
└───────────────────┴────────────────┴───────────────────┘

1

As it can be seen, vectors and scalars in APL follow the properties of vector spaces. This is a direct consequence of the application of the concepts of [scalar function](https://aplwiki.com/wiki/Scalar_function) and [scalar extension](https://aplwiki.com/wiki/Scalar_extension).

## Nested vectors

APL is an array language, so it can work with several vectors at the same time. There are various ways in which we could choose to represent multiple vectors. We will do it always using rank 1 arrays, but where the elements can be [nested arrays](https://aplwiki.com/wiki/Nested_array). We will call the vectors with non-scalar components *nested vectors*, and each of the vectors that a nested vector contains an *element array*.

For example, we could represent multiple vectors as "vectors of vectors", in which each component (each element array) is a rank 1 array too (we also define an `RD` function to make it easier to generate some random data):

In [4]:
RD ← {5×?⍵⍴0}   ⍝ (R)andom (D)ata
⍝ eg
'ab' Table a b ← ⊂∘RD¨2⍴5   ⍝ 2 random scalars with 5 elements each
'u'  Table ,⊂u ←   RD¨3⍴5   ⍝ random 3D vector with 5 elements by component
'v'  Table ,⊂v ←   RD¨3⍴5   ⍝ random 3D vector with 5 elements by component
'w'  Table ,⊂w ←   RD¨3⍴5   ⍝ random 3D vector with 5 elements by component
a b VS u v w                ⍝ vector space?

┌──────────────────────────────┬─────────────────────────────────┐
│a │b │
├──────────────────────────────┼─────────────────────────────────┤
│┌────────────────────────────┐│┌───────────────────────────────┐│
││1.098 1.484 2.35 4.219 1.255│││2.466 0.4856 0.9174 3.772 2.152││
│└────────────────────────────┘│└───────────────────────────────┘│
└──────────────────────────────┴─────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│u │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│┌─────────────────────────────┬────────────────────────────────┬────────────────────────────┐│
││3.378 1.716 1.24 4.954 0.3482│2.169 0.9802 1.121 0.01969 4.075│1.46 2.968 2.539 1.142 2.271││
│└─────────────────────────────┴────────────────────────────────┴────────────────────────────┘│
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────────────────────────────────┐
│v │
├───────────────────────────────────────────────────────────────────────────────────────────────┤
│┌─────────────────────────────┬────────────────────────────────┬──────────────────────────────┐│
││4.614 3.694 4.188 1.685 4.103│1.983 2.312 2.903 1.371 0.007588│0.2758 3.486 3.584 2.215 3.317││
│└─────────────────────────────┴────────────────────────────────┴──────────────────────────────┘│
└───────────────────────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│w │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│┌───────────────────────────────┬──────────────────────────────┬─────────────────────────────┐│
││1.218 4.083 3.279 0.06128 0.206│0.6199 1.765 3.098 1.303 2.458│2.766 4.419 2.85 0.6335 3.176││
│└───────────────────────────────┴──────────────────────────────┴─────────────────────────────┘│
└──────────────────────────────────────────────────────────────────────────────────────────────┘

1

The vector space properties stand for APL nested vectors too. Of course, only as long as their elements [conform](https://aplwiki.com/wiki/Conformability). Vectors and scalars must conform too. Therefore, we need to enclose `a` and `b` to actually be scalars, and the `u`, `v` and `w` vectors must be rank-1 arrays of the same length:

In [5]:
(,'⍴' '⍴¨'∘.,'abuvw') Table (⍴¨,⍴¨¨)a b u v w
⍝ checks in VS:
Assert (⊂⍬) ≡∘⍴¨ a b        ⍝ a and b are scalars
Assert (⊂,1) ≡∘⍴∘⍴¨ u v w   ⍝ u, v and w are vectors (rank 1 arrays)
Assert 2=/≢¨ u v w          ⍝ u, v and w have the same length

┌──┬──┬──┬──┬──┬───┬───┬───────┬───────┬───────┐
│⍴a│⍴b│⍴u│⍴v│⍴w│⍴¨a│⍴¨b│⍴¨u │⍴¨v │⍴¨w │
├──┼──┼──┼──┼──┼───┼───┼───────┼───────┼───────┤
│ │ │3 │3 │3 │┌─┐│┌─┐│┌─┬─┬─┐│┌─┬─┬─┐│┌─┬─┬─┐│
│ │ │ │ │ ││5│││5│││5│5│5│││5│5│5│││5│5│5││
│ │ │ │ │ │└─┘│└─┘│└─┴─┴─┘│└─┴─┴─┘│└─┴─┴─┘│
└──┴──┴──┴──┴──┴───┴───┴───────┴───────┴───────┘

Other representations of multiple vectors are also possible. Just to display them in a convenient way, we will use "vectors of columns" (and define an `RC` function analogous to `RD` to generate random columns):

In [6]:
RC ← (⍪RD)¨⍴   ⍝ generate ⍺ (R)andom (C)olumns of ⍵ rows
⍝ eg
'ab'  Table a b   ← ⊂¨ 2 RC 5   ⍝ 2 random scalar columns of 5 rows
'uvw' Table u v w ← ↓3 3 RC 5   ⍝ 3 random 3D vectors of 5 rows
a b VS u v w                    ⍝ vector space?

┌────────┬───────┐
│a │b │
├────────┼───────┤
│┌──────┐│┌─────┐│
││3.794 │││3.535││
││0.2032│││4.926││
││3.971 │││4.397││
││2.502 │││4.684││
││1.006 │││4.943││
│└──────┘│└─────┘│
└────────┴───────┘

┌────────────────────┬─────────────────────┬─────────────────────┐
│u │v │w │
├────────────────────┼─────────────────────┼─────────────────────┤
│┌─────┬─────┬──────┐│┌─────┬──────┬──────┐│┌─────┬───────┬─────┐│
││4.273│4.689│0.6168│││2.091│2.395 │2.965 │││1.306│4.099 │1.667││
││1.203│3.975│2.405 │││2.992│1.062 │1.84 │││1.064│3.998 │1.804││
││1.929│1.275│0.9094│││2.048│0.2925│0.5153│││3.828│0.9132 │2.195││
││1.21 │2.614│4.783 │││1.754│2.754 │4.472 │││2.859│1.289 │1.014││
││1.23 │4.212│2.414 │││1.887│2.398 │1.623 │││1.038│0.01302│1.576││
│└─────┴─────┴──────┘│└─────┴──────┴──────┘│└─────┴───────┴─────┘│
└────────────────────┴─────────────────────┴─────────────────────┘

1

Vectors of columns form a vector space too. The shape is the same as before, and the elements conform:

In [7]:
(,'⍴' '⍴¨'∘.,'abuvw') Table (⍴¨,⍴¨¨)a b u v w

┌──┬──┬──┬──┬──┬─────┬─────┬─────────────┬─────────────┬─────────────┐
│⍴a│⍴b│⍴u│⍴v│⍴w│⍴¨a │⍴¨b │⍴¨u │⍴¨v │⍴¨w │
├──┼──┼──┼──┼──┼─────┼─────┼─────────────┼─────────────┼─────────────┤
│ │ │3 │3 │3 │┌───┐│┌───┐│┌───┬───┬───┐│┌───┬───┬───┐│┌───┬───┬───┐│
│ │ │ │ │ ││5 1│││5 1│││5 1│5 1│5 1│││5 1│5 1│5 1│││5 1│5 1│5 1││
│ │ │ │ │ │└───┘│└───┘│└───┴───┴───┘│└───┴───┴───┘│└───┴───┴───┘│
└──┴──┴──┴──┴──┴─────┴─────┴─────────────┴─────────────┴─────────────┘

In some situations, it can be useful to represent multiple vectors not as a nested vector, but as an array of vectors, with the shape of the element arrays. The function `AV` takes a nested vector and returns an array of vectors, while `VA` does the opposite operation:

In [8]:
AV ← ⊂⍤ 1↑[¯0.5]   ⍝ array of vectors from vector
VA ← ⊂⍤¯1↑[¯0.5]   ⍝ vector from array of vectors
Assert v ≡ VA AV v
'v' '⍴v' '⍴¨v' 'AV v' '⍴AV v' '⍴¨AV v' Table v (⍴v) (⍴¨v) (AV v) (⍴AV v) (⍴¨AV v)

┌─────────────────────┬──┬─────────────┬─────────────────────┬─────┬──────┐
│v │⍴v│⍴¨v │AV v │⍴AV v│⍴¨AV v│
├─────────────────────┼──┼─────────────┼─────────────────────┼─────┼──────┤
│┌─────┬──────┬──────┐│3 │┌───┬───┬───┐│┌───────────────────┐│5 1 │┌─┐ │
││2.091│2.395 │2.965 ││ ││5 1│5 1│5 1│││2.091 2.395 2.965 ││ ││3│ │
││2.992│1.062 │1.84 ││ │└───┴───┴───┘│├───────────────────┤│ │├─┤ │
││2.048│0.2925│0.5153││ │ ││2.992 1.062 1.84 ││ ││3│ │
││1.754│2.754 │4.472 ││ │ │├───────────────────┤│ │├─┤ │
││1.887│2.398 │1.623 ││ │ ││2.048 0.2925 0.5153││ ││3│ │
│└─────┴──────┴──────┘│ │ │├───────────────────┤│ │├─┤ │
│ │ │ ││1.754 2.754 4.472 ││ ││3│ │
│ │ │ │├───────────────────┤│ │├─┤ │
│ │ │ ││1.887 2.398 1.623 ││ ││3│ │
│ │ │ │└───────────────────┘│ │└─┘ │
└─────────────────────┴──┴─────────────┴─────────────────────┴─────┴──────┘

## Scalar extension

If we are working with nested vectors, it could often happen that all the elements of some element array have the same value. For instance, a component could take the value zero if all vectors are contained in a plane. In these cases, we do not need to store multiple copies of the same value. After all, APL will use scalar extension if we have some scalar component:

In [9]:
u[1]   ← 0            ⍝ vectors parallel to plane xz
w[0 2] ← 0 1          ⍝ vectors parallel to direction y
'uvw' Table u v w     ⍝ vectors with scalar 0 components
a b VS u v w          ⍝ vector space?

┌────────────────┬─────────────────────┬─────────────┐
│u │v │w │
├────────────────┼─────────────────────┼─────────────┤
│┌─────┬─┬──────┐│┌─────┬──────┬──────┐│┌─┬───────┬─┐│
││4.273│0│0.6168│││2.091│2.395 │2.965 │││0│4.099 │1││
││1.203│ │2.405 │││2.992│1.062 │1.84 │││ │3.998 │ ││
││1.929│ │0.9094│││2.048│0.2925│0.5153│││ │0.9132 │ ││
││1.21 │ │4.783 │││1.754│2.754 │4.472 │││ │1.289 │ ││
││1.23 │ │2.414 │││1.887│2.398 │1.623 │││ │0.01302│ ││
│└─────┴─┴──────┘│└─────┴──────┴──────┘│└─┴───────┴─┘│
└────────────────┴─────────────────────┴─────────────┘

1

Vectors with mixed array and scalar components also follow the properties of vector spaces. Some components have different shapes, but all of them conform:

In [10]:
(,'⍴' '⍴¨'∘.,'uvw') Table (⍴¨,⍴¨¨)u v w

┌──┬──┬──┬──────────┬─────────────┬───────┐
│⍴u│⍴v│⍴w│⍴¨u │⍴¨v │⍴¨w │
├──┼──┼──┼──────────┼─────────────┼───────┤
│3 │3 │3 │┌───┬┬───┐│┌───┬───┬───┐│┌┬───┬┐│
│ │ │ ││5 1││5 1│││5 1│5 1│5 1││││5 1│││
│ │ │ │└───┴┴───┘│└───┴───┴───┘│└┴───┴┘│
└──┴──┴──┴──────────┴─────────────┴───────┘

Vectors with scalar components of course are smaller:

In [11]:
]SizeOf u v w

v 296 u 248 w 200

In some cases, this space savings could be significant. However, either keeping the size low when working with the vector or simplifying it later will have a performance cost. In following sections, we will study the performance and memory usage implications of vectors with scalar components with more detail.

To make it easier working with vectors with scalar components, we define the collapse function, `C`, and the zero operator, `_Z`. The `C` function will colapse components of nested vectors into scalars if all elements are equal. The `⎕CT` value used in the comparison can be passed as left argument. The function makes sure that all values for which `⎕CT>|` is true are treated as zero (in general, APL will never consider identical two values when one of them is `0`, independently of `⎕CT`). For completeness, an inverse function `US` is also included. The `_Z` operator is intended to be used with dyadic functions for which a zero argument implies a result of zero, such as multiplication. It checks if any of the arguments is a scalar `0` and, in that case, it returns a scalar `0`. This way we avoid having to run `C` later, which is relatively expensive. Consequently, the performance is improved when using vectors with scalar zeros. It is still lower than when not using `C` or `_Z` at all, but then the space requirements will be higher:

In [12]:
 C  ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ 1=≢∪⍵:⊃⍵ ⋄ ⍺∧.>|∊⍵:0 ⋄ ⍵}¨   ⍝ (C)ollapse identical values into scalar
_Z  ← {0≡⍵:0 ⋄ 0≡⍺:0 ⋄ ⍺(⍺⍺)⍵}                      ⍝ result (Z)ero for zero argument
⍝ eg
v1 v2 ← ((1 2) 0 (3 4) (5 6)) ((7 8) (9 10) (11 12) 0)
'v1' 'v2' 'v1 × v2' 'v1 ×_Z¨ v2' Table v1 v2 (v1 × v2) (v1 ×_Z¨ v2)
Assert 0 1 2 3   ≡ C 0 1 (2 2 2) 3
Assert (0 1) 0 ≡ (0 1) (2 3) ×_Z¨ 1 0
]Runtime '  u   ×¨   w' 'C u   ×¨   w' '  u   ×_Z¨ w' -compare
]Runtime 'r1←  u ∘.×¨   w' 'r2←C u ∘.×¨   w' 'r3←  u ∘.×_Z¨ w' -compare
'r1' 'r2' 'r3' Table r1 r2 r3
]SizeOf r1 r2 r3

┌───────────────┬──────────────────┬────────────────────┬────────────────┐
│v1 │v2 │v1 × v2 │v1 ×_Z¨ v2 │
├───────────────┼──────────────────┼────────────────────┼────────────────┤
│┌───┬─┬───┬───┐│┌───┬────┬─────┬─┐│┌────┬───┬─────┬───┐│┌────┬─┬─────┬─┐│
││1 2│0│3 4│5 6│││7 8│9 10│11 12│0│││7 16│0 0│33 48│0 0│││7 16│0│33 48│0││
│└───┴─┴───┴───┘│└───┴────┴─────┴─┘│└────┴───┴─────┴───┘│└────┴─┴─────┴─┘│
└───────────────┴──────────────────┴────────────────────┴────────────────┘

u ×¨ w → 2.7E¯7 | 0% ⎕⎕⎕ 
* C u ×¨ w → 3.3E¯6 | +1149% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u ×_Z¨ w → 7.6E¯7 | +184% ⎕⎕⎕⎕⎕⎕⎕⎕⎕

r1← u ∘.×¨ w → 6.4E¯7 | 0% ⎕⎕⎕⎕⎕⎕ 
* r2←C u ∘.×¨ w → 4.3E¯6 | +566% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* r3← u ∘.×_Z¨ w → 9.6E¯7 | +50% ⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌────────────┬────────────┬────────────┐
│r1 │r2 │r3 │
├────────────┼────────────┼────────────┤
│┌─┬─┬──────┐│┌─┬─┬──────┐│┌─┬─┬──────┐│
││0│0│0.6168│││0│0│0.6168│││0│0│0.6168││
││0│0│2.405 │││ │ │2.405 │││ │ │2.405 ││
││0│0│0.9094│││ │ │0.9094│││ │ │0.9094││
││0│0│4.783 │││ │ │4.783 │││ │ │4.783 ││
││0│0│2.414 │││ │ │2.414 │││ │ │2.414 ││
│└─┴─┴──────┘│└─┴─┴──────┘│└─┴─┴──────┘│
└────────────┴────────────┴────────────┘

r1 232 r2 200 r3 200

## Vector extension

Actually, there is no reason to require vectors to be of the same length to do operations with them. A vector of `n` dimensions can always be interpreted as a vector of `n+1` dimensions in which the additional component is `0` (and, therefore, also a vector of `n+2` components, `n+3`, and so on). However, if we try to operate with vectors of different lengths in APL, we will get an error:

In [13]:
1 2 3 + 4 5   ⍝ ERROR: length

LENGTH ERROR: Mismatched left and right argument shapes
      1 2 3+4 5   ⍝ ERROR: length
           ∧


We are going define some functions and operators to make it easier to work with vectors of different lengths. We want to be able to extend the arguments of functions to be of the required length, appending as many (scalar) zeros as needed. We will also want to remove trailing zeros when they are not needed.

In [14]:
 X  ← {⍺↑⍵,0⍴⍨0⌈⍺-≢⍵}                        ⍝ e(X)tend vector ⍵ to length ⍺ appending zeros
_X_ ← {⍺←⊢ ⋄ ⍺ ⍺⍺⍥((⍺(⍵⍵⍨⍨)⍵)∘X)⍵}           ⍝ e(X)tend ⍺⍺ arguments to have length given by ⍵⍵
_X  ← _X_(⌈⍥≢)                               ⍝ e(X)tend ⍺⍺ arguments to have same (maximum) length
_ZX ← {⍺ ⍺⍺_Z¨_X_(⌊⍥≢)⍵}                     ⍝ e(X)tend ⍺⍺ arguments to have same (minimum) length and _Z¨
 S  ← {⍬≡⍵:0 ⋄ (1<≢⍵)∨(0<≡⍵)∧1<≢⊃⍵:⍵ ⋄ ⊃⍵}   ⍝ return (S)calar as scalar
 Z  ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ S⍵↓⍨-⊥⍨(1∧.=1+∊)¨⍵}   ⍝ remove trailing (Z)eros
⍝ eg
Assert 0 1 2 0   ≡ 4 X 0 1 2
Assert 0 1       ≡ 2 X 0 1 2
Assert 0 1 2     ≡ Z 0 1 2 (0 0 0) 0
Assert 0         ≡ Z 0 0 0
Assert 1         ≡ Z 1 0
Assert 0 1 1     ≡ 0 0 1 + _X 0 1
Assert 1 0 2     ≡ Z 1 0 2 0 0
Assert 1 0 2     ≡ 1 0 2 3 ×_ZX 1 1 1

The function `X` extends a vector with scalar zeros up to the given length. The `_X_` operator applies the given function after extending the arguments to the length specified, either explicitly or as the result of a function. For instance, the monadic operator `_X` extend the arguments up to the maximum length of both of them (`⌈⍥≢`), while `_ZX` will "extend" them to the minimum length and apply the given function using `_Z¨`. The functions `S` and `Z` are used to simplify vectors: `S` will return 1-element vectors as scalars and 0-elements vectors as `0`, while `Z` will remove trailing zeros (like `C`, using `⎕CT` or a left argument), including non-scalar components in which all elements are zero.

When the arguments of a function are first extended to the required length, and later the result is simplified collapsing into scalars and removing trailing zeros, we will say that the function is applied using *vector extension*. The operator `_V` applies a dyadic function using vector extension.

In [15]:
_V  ← {⍺←⊢ ⋄ Z∘C⍺(⍺⍺_X)⍵}                               ⍝ (V)ector extension
⍝ eg
v1 v2 ← (1 2 3) (4 5)
'v1' 'v2' 'v1 +_V v2' Table v1 v2 (v1 +_V v2)           ⍝ no error now
v1 v2 ← ((⍪v1) 0 0 (⍪v1) 0) ((⍪v2,6) (⍪v2,6) 0 (⍪v1))
'v1' 'v2' 'v1 -_V v2' Table v1 v2 (v1 -_V v2)           ⍝ subtraction under vector extension
Assert 0 (¯3 ¯4) ≡ 0 0 0 (1 2) -_V 0 (3 4) 0 (1 2)

┌─────┬───┬─────────┐
│v1 │v2 │v1 +_V v2│
├─────┼───┼─────────┤
│1 2 3│4 5│5 7 3 │
└─────┴───┴─────────┘

┌───────────┬─────────┬─────────┐
│v1 │v2 │v1 -_V v2│
├───────────┼─────────┼─────────┤
│┌─┬─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌──┬──┐ │
││1│0│0│1│0│││4│4│0│1│││¯3│¯4│ │
││2│ │ │2│ │││5│5│ │2│││ │¯5│ │
││3│ │ │3│ │││6│6│ │3│││ │¯6│ │
│└─┴─┴─┴─┴─┘│└─┴─┴─┴─┘│└──┴──┘ │
└───────────┴─────────┴─────────┘

## Vector spaces under vector extension

Now that we can operate with vectors of arbitrary lengths, we can redefine the `VS` function, removing the requirement for vectors to have the same length. We also define an `E` function to check that two vectors are *equivalent* or, in other words, that they match after being simplified by `Z∘C`.

In [16]:
E ← ≡⍥(Z∘C)   ⍝ (E)quivalent vectors

In [17]:
]dinput
VS ← {                                             ⍝ (V)ector (S)pace
    _ ← Assert (⊂⍬) ∧.≡ ⍴¨ a b ← ⍺                 ⍝ a and b are scalars
    _ ← Assert (⊂,1) ∧.≥ ⍴∘⍴¨ u v w ← ⍵            ⍝ u, v and w are vectors (rank 1)
                                                   ⍝ properties:
    _ ← Assert v                 E v + 0           ⍝   neutral element for addition
    _ ← Assert v                 E 1 × v           ⍝   neutral element for multiplication
    _ ← Assert 0                 E v +_V -v        ⍝   inverse element for addition
    _ ← Assert (v -_V w)         E v +_V -w        ⍝   substraction function
    _ ← Assert (v +_V w)         E w +_V v         ⍝   commutative addition
    _ ← Assert (a × v)           E v × a           ⍝   commutative product with scalars
    _ ← Assert (u +_V v +_V w)   E w +_V u +_V v   ⍝   associative addition
    _ ← Assert (v × a × b)       E (v × a) × b     ⍝   associative product with scalars
    _ ← Assert (a × v +_V w)     E v +_V⍥(a∘×) w   ⍝   distributive over addition of vectors
    _ ← Assert (v × a + b)       E a +_V⍥(v∘×) b   ⍝   distributive over addition of scalars
    1
}

In [18]:
a b VS 1 (0 1) (0 0 1)

1

## Magnitude and unitary vector

Let's define some more functions to work with vectors. The function `M` calculates the magnitude of a vector using the dot product, while `U` returns a unitary vector parallel to the given one dividing it by `M`.

In [19]:
M ← 0.5*⍨+.×⍨   ⍝ (M)agnitude of vector
U ← ÷∘M⍨        ⍝ (U)nitary vector
⍝ eg
'M v' 'U v' 'M w' 'U w' Table (M v) (U _V v) (M w) (U _V w)
Assert E∘(M×U)⍨¨ u v w   ⍝ a vector is the product of its magnitude by its unit vector

┌───────┬──────────────────────┬───────┬──────────────────┐
│M v │U v │M w │U w │
├───────┼──────────────────────┼───────┼──────────────────┤
│┌─────┐│┌──────┬──────┬──────┐│┌─────┐│┌─┬───────┬──────┐│
││4.348│││0.481 │0.5509│0.682 │││4.22 │││0│0.9715 │0.237 ││
││3.67 │││0.8153│0.2894│0.5015│││4.121│││ │0.9701 │0.2427││
││2.132│││0.9606│0.1372│0.2418│││1.354│││ │0.6743 │0.7384││
││5.538│││0.3168│0.4974│0.8076│││1.632│││ │0.7901 │0.6129││
││3.456│││0.5459│0.6939│0.4696│││1 │││ │0.01302│0.9999││
│└─────┘│└──────┴──────┴──────┘│└─────┘│└─┴───────┴──────┘│
└───────┴──────────────────────┴───────┴──────────────────┘

# Geometric product

## Inverse of a vector

In addition to the properties of vector spaces, in a geometric algebra, there is an inverse element for multiplication using the geometric product (which has not been defined yet) such that for every vector `v` not equivalent to zero exists an inverse, that multiplied by the original vector results in `1`.

It can help at this point to think of a concrete example to get an intuitive idea of what this inverse should be. Let's consider, for instance, a velocity, a scalar velocity. Let’s say this velocity is $5\,m/s$. There is no discussion in that the inverse of this velocity is $0.2\,s/m$. Similarly, a velocity of $-5\,m/s$ will have an inverse of $-0.2\,s/m$. The direction of the inverse is the same as the original, but its magnitude is the inverse of the original magnitude. Another way to express this is that the inverse is the original value divided by the square of its magnitude:

In [20]:
Assert ((|∘÷ ≡ ÷∘|),(×∘÷ ≡ ×),(÷ ≡ ÷∘(×⍨|)⍨))5 ¯5
' ' '÷' '÷∘(×⍨|)⍨' '×∘÷⍨' Table (5 ¯5) (÷5 ¯5) (÷∘(×⍨|)⍨5 ¯5) (×∘÷⍨5 ¯5)

┌────┬────────┬────────┬────┐
│ │÷ │÷∘(×⍨|)⍨│×∘÷⍨│
├────┼────────┼────────┼────┤
│5 ¯5│0.2 ¯0.2│0.2 ¯0.2│1 1 │
└────┴────────┴────────┴────┘

Our intention is to extend this concept to vectors. Therefore, we define the inverse of any vector `v` not equivalent to zero (`0(~E)v`) as another vector with the same direction of `v`, but with a magnitude which is the inverse of the magnitude of `v`. As with scalars, we can achieve this result dividing the original vector by the square of its magnitude (which, for vectors, is just its self dot product, `+.×⍨`):

In [21]:
I ← ×∘(÷+.×⍨)⍨   ⍝ (I)nverse vector
⍝ eg
'I v' '+.×∘I⍨ v' 'I w' '+.×∘I⍨ w' Table (I _V v) (+.×∘I⍨ v) (I _V w) (+.×∘I⍨ w) 
Assert (M I v) ≡ ÷ M v        ⍝ magnitude of inverse is inverse of magnitude
Assert (U I v) ≡ U v          ⍝ same direction
Assert   (I v) ≡ ((÷M)×U) v   ⍝ inverse vector
⍝ the inverse of v is v divided by the square of its magnitude:
Assert   (I v) ≡ ÷∘(×⍨M)⍨ v   ⍝ analogous to (÷⍵) ≡ ÷∘(×⍨|)⍨⍵

┌───────────────────────┬────────┬───────────────────┬────────┐
│I v │+.×∘I⍨ v│I w │+.×∘I⍨ w│
├───────────────────────┼────────┼───────────────────┼────────┤
│┌──────┬───────┬──────┐│┌─┐ │┌─┬───────┬───────┐│┌─┐ │
││0.1106│0.1267 │0.1569│││1│ ││0│0.2302 │0.05616│││1│ │
││0.2222│0.07886│0.1367│││1│ ││ │0.2354 │0.05889│││1│ │
││0.4506│0.06438│0.1134│││1│ ││ │0.4979 │0.5453 │││1│ │
││0.0572│0.08982│0.1458│││1│ ││ │0.4843 │0.3757 │││1│ │
││0.1579│0.2008 │0.1359│││1│ ││ │0.01302│0.9998 │││1│ │
│└──────┴───────┴──────┘│└─┘ │└─┴───────┴───────┘│└─┘ │
└───────────────────────┴────────┴───────────────────┴────────┘

This definition might look a bit arbitrary, and indeed it is up to some point. We will come back to it later.

Notice that the `I` function does not directly uses the dyadic divide function because, if we defined `I` as `÷∘(+.×⍨)⍨`, when we had a vector equivalent by zero, we would be dividing `0` by `0` and, in APL, `1 ≡ 0÷0`. However `÷0` results in a domain error, which looks like a more reasonable response. It is also important to be careful with nested vectors, since the function will fail with a single vector equivalent to `0` (when `0(~E)¨AV`):

In [22]:
Assert 0(~E)¨AV v    ⍝ v has an inverse
vz←v ⋄ (3⌷¨vz) ← 0   ⍝ vz is z with a 0 vector
(⊂'vz') Table ,⊂vz
Assert 0(~E)¨AV vz   ⍝ FAIL
I vz                 ⍝ ERROR: domain

┌─────────────────────┐
│vz │
├─────────────────────┤
│┌─────┬──────┬──────┐│
││2.091│2.395 │2.965 ││
││2.992│1.062 │1.84 ││
││2.048│0.2925│0.5153││
││0 │0 │0 ││
││1.887│2.398 │1.623 ││
│└─────┴──────┴──────┘│
└─────────────────────┘

assertion failure
      Assert 0(~E)¨AV vz   ⍝ FAIL
      ∧
DOMAIN ERROR: Divide by zero
      I vz                 ⍝ ERROR: domain
      ∧


## Geometric product of vectors

Once vector inverses are defined, the geometric product of vectors (represented here by `∆V`) is the product for which a vector multiplied by its inverse is equal to one:

    1 ≡ v ∆V I v  ⍝ ∆V is the geometric product of vectors
    
From this definition, several properties can be derived. Before defining the `∆V` function, let's first see what is the result of multiplying parallel and perpendicular vectors. We will later use these results to define arbitrary geometric products.

### Geometric product of parallel vectors

When we calculate the inverse of a unitary vector using our definition, we will find that its inverse is the same vector. We divide by 1 when dividing by its magnitude (or its squared magnitude), leaving the original vector unchanged:

In [23]:
Assert ≡∘I⍨ U v
'U v' 'I U v' Table (U v) (I U v)

┌──────────────────────┬──────────────────────┐
│U v │I U v │
├──────────────────────┼──────────────────────┤
│┌──────┬──────┬──────┐│┌──────┬──────┬──────┐│
││0.481 │0.5509│0.682 │││0.481 │0.5509│0.682 ││
││0.8153│0.2894│0.5015│││0.8153│0.2894│0.5015││
││0.9606│0.1372│0.2418│││0.9606│0.1372│0.2418││
││0.3168│0.4974│0.8076│││0.3168│0.4974│0.8076││
││0.5459│0.6939│0.4696│││0.5459│0.6939│0.4696││
│└──────┴──────┴──────┘│└──────┴──────┴──────┘│
└──────────────────────┴──────────────────────┘

Also by definition, the geometric product of a vector by its inverse must be 1. This implies that multiplying a unitary vector by itself will also be 1:

    1 ≡ ∆V⍨ U v
    
More generally, we can calculate what is the result of multiplying any vector by itself. If a vector `v` is defined as `m×u`, where `m` is a scalar and `u` is a unitary vector, we can define its square as:

    (∆V⍨ v) ≡ (m × u) ∆V m × u   ⍝ v ≡ m × u
    (∆V⍨ v) ≡ m × m × u ∆V u     ⍝ (a × v) ≡ v × a
    (∆V⍨ v) ≡ m × m              ⍝ 1 ≡ ∆V⍨ u

The result of squaring a vector is the square of its magnitude. To multiply two parallel vectors `v` and `w` defined, respectively, as `a×u` and `b×u`:

    (v ∆V w) ≡ (a × u) ∆V b × u
    (v ∆V w) ≡ a × b × u ∆V u
    (v ∆V w) ≡ a × b
    
So, the product of two parallel vectors is the scalar that results from multiplying their magnitudes.

### Geometric product of perpendicular vectors

Every vector can be descomposed in two perpendicular components in some base. For example:

In [24]:
v1 v2 ← 3 2   ⍝ eg
Assert (⊃ v1 v2 +.× (1 0) (0 1)) ≡ (M×U) v1 v2   ⍝ ortoghonal components or magnitude and unit vector
Assert 0 ≡ v1 0 +.× 0 v2                         ⍝ (v1 0) and (0 v2) are perpendicular (dot product is zero)
'v' 'v1' 'u1' 'v2' 'u2' Table (3 2) 3 (1 0) 2 (0 1)
'v1 0 +.× 0 v2' 'm ← M v' 'u ← U v' 'm × u' Table (3 0+.×0 2) (M v1 v2) (U v1 v2) ((M×U) v1 v2)

┌───┬──┬───┬──┬───┐
│v │v1│u1 │v2│u2 │
├───┼──┼───┼──┼───┤
│3 2│3 │1 0│2 │0 1│
└───┴──┴───┴──┴───┘

┌─────────────┬───────┬─────────────┬─────┐
│v1 0 +.× 0 v2│m ← M v│u ← U v │m × u│
├─────────────┼───────┼─────────────┼─────┤
│0 │3.606 │0.8321 0.5547│3 2 │
└─────────────┴───────┴─────────────┴─────┘

We saw before that the result of squaring the vector `v` (with magnitude `m`) will be its magnitude squared (`×⍨m`). We must get the same result when using as starting point the decomposition of `v` in perpendicular vectors:

    (∆V⍨ v) ≡ ×⍨m
    (∆V⍨ v) ≡ ((v1×u1) + v2×u2) ∆V (v1×u1) + v2×u2

Developing further this expression, applying the distributive property:

    (∆V⍨ v) ≡ (∆V⍨ v1×u1) + ((v1×u1) ∆V v2×u2) + ((v2×u2) ∆V v1×u1) + ∆V⍨ v2×u2

And, applying the already known square formula:

    (∆V⍨ v) ≡ (×⍨v1) + (v1×v2 × u1 ∆V u2) + (v1×v2 × u2 ∆V u1) + ×⍨v2

The value `∆V⍨ v` is also `×⍨m` which, according to [Pythagoras](https://en.wikipedia.org/wiki/Pythagorean_theorem), must be equal to `(×⍨v1) + ×⍨v2`. Therefore:

    0 ≡ v1×v2 × (u1 ∆V u2) + u2 ∆V u1
    0 ≡ (u1 ∆V u2) + u2 ∆V u1

So:

    (u1 ∆V u2) ≡ -u2 ∆V u1

The product of two perpendicular vectors is anti-commutative. As a particular solution, both `u1 ∆V u2` and `u2 ∆V u2` might be equal to zero. Let's assume they do and see what happens:

    (V u) ≡ (V u) ∆ (V v) ∆ (V v)
    (V u) ≡((V u) ∆ (V v))∆ (V v)
    (V u) ≡             0 ∆ (V v)
    (V u) ≡ 0 ⍝ ???
    
    

    (V u) ≡ (V u) ∆ (V v) ∆ (V v)
    (V u) ≡ (V v) ∆⍨(V u) ∆ (V v)
    0 ≢ (V u) ∆ (V v)

 If we defined the product of orthogonal vectors as zero, we would find that the product we are searching for is the dot product (`∆V ← +.×`). Instead, in geometric algebras we will impose that the geometric product of two perpendiculars vectors must be different from zero:

    (0 ≢ u +.× v) ∨ 0 ≢ u ∆V v   ⍝ in other words: u (+.×∨⍥(0∘≢)∆V) v
    
As we will see, this decission has important consequences.

### Bivectors

It can be convenient, to get an intuitive idea of the geometric properties of multivectors to think in a more tangible example.

We may have, for instance, two perpendicular vectors, each of them representing the width and height of a sheet of paper. To calculate the area of the sheet, we calculate the geometric product `h ∆V w`. Since `h` and `w` are perpendicular, the result will be a bivector (the bivector `hw`). It is easy to prove that we will obtain the same result independently of any rotation that we apply to the sheet of paper.

In [25]:
⍝ figure
ps ls←(5 6⍴'h',¯29↑'0    w')(3 9⍴'0',(¯8↑'h'),¯18↑9↑'w')                   ⍝ portrait and landscape sheets
r←' ' '○0.5' '○1' '○1.5' Table ⊂¨ps ls (⌽⊖ps) (⌽⊖ls)
r⊣r⍪←'0 h ∆V w 0' 'h 0 ∆V 0 ¯w' '0 ¯h ∆V ¯w 0' '¯h 0 ∆V 0 w' Table ⊂'hw'   ⍝ rotated

┌──────────┬───────────┬────────────┬───────────┐
│ │○0.5 │○1 │○1.5 │
├──────────┼───────────┼────────────┼───────────┤
│┌──────┐ │┌─────────┐│┌──────┐ │┌─────────┐│
││h │ ││0 h│││w 0│ ││ w││
││ │ ││ │││ │ ││ ││
││ │ ││w │││ │ ││h 0││
││ │ │└─────────┘││ │ │└─────────┘│
││0 w│ │ ││ h│ │ │
│└──────┘ │ │└──────┘ │ │
├──────────┼───────────┼────────────┼───────────┤
│0 h ∆V w 0│h 0 ∆V 0 ¯w│0 ¯h ∆V ¯w 0│¯h 0 ∆V 0 w│
├──────────┼───────────┼────────────┼───────────┤
│hw │hw │hw │hw │
└──────────┴───────────┴────────────┴───────────┘

However, if we flip the sheet, the geometric product will be negative!

In [26]:
⍝ figure
f←('⊖',¨' ' '○0.5' '○1' '○1.5') Table ⊂∘⊖¨ps ls (⌽⊖ps) (⌽⊖ls)
f⊣f⍪←'0 ¯h ∆V w 0' 'h 0 ∆V 0 w' '0 h ∆V ¯w 0' '¯h 0 ∆V 0 ¯w' Table ⊂'-hw'  ⍝ flipped vertically

┌───────────┬───────────┬───────────┬────────────┐
│⊖ │⊖○0.5 │⊖○1 │⊖○1.5 │
├───────────┼───────────┼───────────┼────────────┤
│┌──────┐ │┌─────────┐│┌──────┐ │┌─────────┐ │
││0 w│ ││w │││ h│ ││h 0│ │
││ │ ││ │││ │ ││ │ │
││ │ ││0 h│││ │ ││ w│ │
││ │ │└─────────┘││ │ │└─────────┘ │
││h │ │ ││w 0│ │ │
│└──────┘ │ │└──────┘ │ │
├───────────┼───────────┼───────────┼────────────┤
│0 ¯h ∆V w 0│h 0 ∆V 0 w │0 h ∆V ¯w 0│¯h 0 ∆V 0 ¯w│
├───────────┼───────────┼───────────┼────────────┤
│-hw │-hw │-hw │-hw │
└───────────┴───────────┴───────────┴────────────┘

The conclusion is the same if we flip the sheet horizontally instead of vertically:

In [27]:
⍝ figure
r,h←('⌽',¨'○1.5' '○1' '○0.5' ' ')⍪(⊂1+⍳3)⌷2⌽⌽f  ⍝ flipped horizontally

┌──────────┬───────────┬────────────┬───────────┬───────────┬───────────┬────────────┬───────────┐
│ │○0.5 │○1 │○1.5 │⌽○1.5 │⌽○1 │⌽○0.5 │⌽ │
├──────────┼───────────┼────────────┼───────────┼───────────┼───────────┼────────────┼───────────┤
│┌──────┐ │┌─────────┐│┌──────┐ │┌─────────┐│┌─────────┐│┌──────┐ │┌─────────┐ │┌──────┐ │
││h │ ││0 h│││w 0│ ││ w│││w │││0 w│ ││h 0│ ││ h│ │
││ │ ││ │││ │ ││ │││ │││ │ ││ │ ││ │ │
││ │ ││w │││ │ ││h 0│││0 h│││ │ ││ w│ ││ │ │
││ │ │└─────────┘││ │ │└─────────┘│└─────────┘││ │ │└─────────┘ ││ │ │
││0 w│ │ ││ h│ │ │ ││h │ │ ││w 0│ │
│└──────┘ │ │└──────┘ │ │ │└──────┘ │ │└──────┘ │
├──────────┼───────────┼────────────┼───────────┼───────────┼───────────┼────────────┼───────────┤
│0 h ∆V w 0│h 0 ∆V 0 ¯w│0 ¯h ∆V ¯w 0│¯h 0 ∆V 0 w│h 0 ∆V 0 w │0 ¯h ∆V w 0│¯h 0 ∆V 0 ¯w│0 h ∆V ¯w 0│
├──────────┼───────────┼────────────┼───────────┼───────────┼───────────┼────────────┼───────────┤
│hw │hw │hw │hw │-hw │-hw │-hw │-hw │
└──────────┴───────────┴────────────┴───────────┴───────────┴───────────┴────────────┴───────────┘

By convention, we decide that a positive area is the one obtained by multiplying height by width (more generally, we impose that the product of a vector of higher dimensions by a vector of lower dimensions is positive). We may as well take the opposite convention and define positive areas as width by height. This would also work but, as it will be clear later, the chosen convention works better with the usual choice of an horizontal axis for one dimension, and APL's right-to-left evaluation rule.

Recall that the product of orthonormal vectors must be anticommutative. Indeed, in our example, flipping the sheet is equivalent to swapping the factors of the product:

In [28]:
(r,h)[;0 4]

┌──────────┬───────────┐
│ │⌽○1.5 │
├──────────┼───────────┤
│┌──────┐ │┌─────────┐│
││h │ ││w ││
││ │ ││ ││
││ │ ││0 h││
││ │ │└─────────┘│
││0 w│ │ │
│└──────┘ │ │
├──────────┼───────────┤
│0 h ∆V w 0│h 0 ∆V 0 w │
├──────────┼───────────┤
│hw │-hw │
└──────────┴───────────┘

This idea can be further extended to higher dimensions. In three dimensions, we will get positive volumes when a positive thickness multiplies a positive area, and negative volumes if one of them is negative. Consequently, a volume is negated when it is turned inside out. For even higher dimensions, the same concepts apply, although it is more difficult to think of intuitive examples.

## Multivectors

The value of the product `y ∆V x` (where `y` and `x` are perpendicular vectors) is not a vector, neither a scalar, it is another entity which we will call a *[bivector](https://en.wikipedia.org/wiki/Bivector)* (eg `yx ← y ∆V x`). Bivectors can be interpreted as oriented planes, just as vectors can be interpreted as oriented segments. If now we take an additional vector `z`, perpendicular to `x` and `y`, and multiply it by the bivector `yx`, the product will be a *[trivector](https://en.wikipedia.org/w/index.php?title=Trivector&redirect=no)* (`zyx`). Trivectors can be interpreted as oriented volumes. More generally, an *[n-vector](https://en.wikipedia.org/wiki/Multivector)* will be the product of a number `n` of vectors perpendicular between them, and can be interpreted as an n-dimensional entity. By extension of this concept, scalars can be considered *0-vectors*.

We will call *multivectors* the elements obtained adding n-vectors. When a multivector is the result of adding n-vectors of the same grade `n`, we will say that it is a *homogeneous multivector* of grade `n` (and it can be proved it is an n-vector too). When a multivector is the sum of n-vectors of different grades (all of them with grade `n≤m`), we will call it a *heterogenous multivector* of grade `m`. Each of the individual components of a heterogeneous multivector are homogeneous multivectors. We will call each of these components a *blade*. Therefore, homogeneous multivectors are multivectors with all blades of the same grade. In summary:

- 0-vector: scalar
- 1-vector: vector (of any number of dimensions)
- 2-vector: product of 2 orthogonal 1-vectors (bivector)
- 3-vector: product of 3 orthogonal 1-vectors (trivector)
- n-vector of grade `n`: product of `n` orthogonal 1-vectors (homogeneous multivector)
- multivector of grade `m`: sum of n-vectors with `n≤m` (heterogeneous multivector)
- blade of grade `n`: each component of a multivector of grade `m` with `m≥n`

It can be proved that multivectors satisfy the properties of vector spaces, including heterogeneous multivectors with blades of different grades. Therefore, we are going to represent multivectors in APL using rank-1 arrays, just as normal vectors.

It will be assumed that the component `n` of an array represents the unit multivector obtained multiplying, from left to right, the set of perpendicular unitary vectors corresponding to the ones in the binary representation of `n` (assuming `⎕IO←0`). Therefore, if the unitary vectors have `n` dimensions, the corresponding multivectors will have `2*n` blades, corresponding to all the possible products between the `n` 1-vectors. The unitary blades in binary form will be called *binary unitary blades*, or bub.

The `BV` function returns the base vectors and `BB` returns the bub, when given the number of components. `G` returns the grade of a multivector. The grade of each blade is found adding the ones in the corresponding bub. The grade of the multivector is the maximum grade of its blades different from zero. Notice that the grade is not the same as the dimensions, given by `D`. While the grade indicates how many 1-vectors are multiplied, the dimensions correspond to the maximum number of dimensions of those 1-vectors. For instance, the vector `z` is a grade-1 blade of 3 dimensions, and the bivector `zy` is a grade-2 blade of 3 dimensions. Therefore, for every blade, `G≤D`.

In [29]:
BV ← S¨1↑⍨¨∘-1+⍳           ⍝ (B)ase unitary (V)ectors for ⍵ components
BB ← ↓∘⍉2⊥⍣¯1⍳             ⍝ (B)inary unitary (B)lades (bub) for ⍵ components
G  ← {⌈/+/¨(0≢¨C⍵)/BB≢⍵}   ⍝ (G)rade
D  ← ⌈2⍟⌈⍥≢                ⍝ (D)imensions
⍝ eg: 3D base vectors
d   ← 3                   ⍝ dimensions (3D)
bu  ← BV 2*d              ⍝ base unitary multivectors
bub ← BB 2*d              ⍝ binary unitary blades
id  ← '1',1↓bub/¨⊂'zyx'   ⍝ names
'unitary blades' '≢' 'bub' 'grade' 'dim' 'name' Table∘(⍪¨) bu (,∘≢¨bu) bub (,∘G¨bu) (,∘D¨bu) id
Assert (G≤D)¨bu

┌─────────────────┬───┬───────┬─────┬───┬─────┐
│unitary blades │≢ │bub │grade│dim│name │
├─────────────────┼───┼───────┼─────┼───┼─────┤
│┌───────────────┐│┌─┐│┌─────┐│┌─┐ │┌─┐│┌───┐│
││1 │││1│││0 0 0│││0│ ││0│││1 ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 1 │││2│││0 0 1│││1│ ││1│││x ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 1 │││3│││0 1 0│││1│ ││2│││y ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 1 │││4│││0 1 1│││2│ ││2│││yx ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 1 │││5│││1 0 0│││1│ ││3│││z ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 0 1 │││6│││1 0 1│││2│ ││3│││zx ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 0 0 1 │││7│││1 1 0│││2│ ││3│││zy ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 0 0 0 1│││8│││1 1 1│││3│ ││3│││zyx││
│└───────────────┘│└─┘│└─────┘│└─┘ │└─┘│└───┘│
└─────────────────┴───┴───────┴─────┴───┴─────┘

The function `V` takes a vector and returns a multivector (when used dyadically, arbitrary components can be specified):

In [30]:
V ← {⍺←2*⍳≢⍵ ⋄ v←0⍴⍨1+⌈/,⍺ ⋄ v[⍺]+←(⍴⍺)↑⍵ ⋄ v}   ⍝ multi(V)ector with blades ⍵ at ⍺ (1-vector by default)
⍝ eg
'V 1 2 3 4' 'V v' '3 5 6 V v' Table (V 1+⍳4) (V v) (3 5 6 V v)
Assert 0 1 2 0 3 ≡ V 1 2 3
Assert 0 0 1 2 0 0 3 ≡ 2 3 6 V 1 2 3

┌─────────────────┬─────────────────────────┬─────────────────────────────┐
│V 1 2 3 4 │V v │3 5 6 V v │
├─────────────────┼─────────────────────────┼─────────────────────────────┤
│0 1 2 0 3 0 0 0 4│┌─┬─────┬──────┬─┬──────┐│┌─┬─┬─┬─────┬─┬──────┬──────┐│
│ ││0│2.091│2.395 │0│2.965 │││0│0│0│2.091│0│2.395 │2.965 ││
│ ││ │2.992│1.062 │ │1.84 │││ │ │ │2.992│ │1.062 │1.84 ││
│ ││ │2.048│0.2925│ │0.5153│││ │ │ │2.048│ │0.2925│0.5153││
│ ││ │1.754│2.754 │ │4.472 │││ │ │ │1.754│ │2.754 │4.472 ││
│ ││ │1.887│2.398 │ │1.623 │││ │ │ │1.887│ │2.398 │1.623 ││
│ │└─┴─────┴──────┴─┴──────┘│└─┴─┴─┴─────┴─┴──────┴──────┘│
└─────────────────┴─────────────────────────┴─────────────────────────────┘

Vectors are created by `V` in two steps. First, a vector of zeros is defined to fit all the given components (with length `1+⌈/,⍺`). Then, that vector is filled adding the elements given as right argument. Notice that, this way, we can also use `V` for sums:

In [31]:
Assert (+.×⍨v) MV (0⊣_X v)V×⍨ v

VALUE ERROR: Undefined name: MV
      Assert(+.×⍨v)MV(0⊣_X v)V×⍨v
                   ∧


We are also going to define a function `B` to get the blades of a multivector grouped according to their grade (calculated as `+/¨BB≢` instead of using `G`). With a left argument, `B` will return the blades of those grades.

In [32]:
B ← {b←Z({⊂⊂⍵}⌸+/¨BB≢⍵)⌷¨⊂,⍵ ⋄ ⍺←⍳≢b ⋄ (⊂⍺)⌷(1+⌈/,⍺)X b}   ⍝ return blades from multivector ⍵
'multivector' 'scalar' 'vector' 'bivector' 'trivector' Table (⊂⍳8),B⍳8
Assert (B⍳8) ≡ (⍳4)(⊃⍤B)¨⊂⍳8

┌───────────────┬──────┬──────┬────────┬─────────┐
│multivector │scalar│vector│bivector│trivector│
├───────────────┼──────┼──────┼────────┼─────────┤
│0 1 2 3 4 5 6 7│0 │1 2 4 │3 5 6 │7 │
└───────────────┴──────┴──────┴────────┴─────────┘

## Geometric product of unit blades

We are now ready to define the geometric product of multivectors, `∆`. First, we will look at the product of unit blades. We know that the result of squaring a unitary vector is `1`, and therefore the product of orthogonal vectors is anticommutative:

    1 ≡ ∆V⍨u                         ⍝ square of unitary vector u is 1
    (u +.× v ≡ 0) ∧ u (∆V⍨≡-⍤∆V) v   ⍝ anticommutative product of orthogonal vectors

When multiplying multivectors, we only need to apply these two rules. For example (we consider the 3D base defined above):

    (yx ∆ x) ≡ (y ∆ x) ∆ x    ⋄  ((y ∆ x) ∆ x) ≡ y ∆ (x ∆ x)      ⋄  (yx ∆ x) ≡  y ∆ 1   ⋄  (yx ∆ x) ≡  y
    (yx ∆ y) ≡ (y ∆ x) ∆ y    ⋄  ((y ∆ x) ∆ y) ≡ (-x ∆ y) ∆ y     ⋄  (yx ∆ y) ≡ -x ∆ 1   ⋄  (yx ∆ y) ≡ -x
    (∆⍨ yx)  ≡ y ∆ x ∆ y ∆ x  ⋄  (∆⍨ yx)       ≡ - y ∆ x ∆ x ∆ y  ⋄  (∆⍨ yx)  ≡ - y ∆ y  ⋄  (∆⍨ yx)  ≡ ¯1 

The same rules apply for more dimensions:

    (zyx ∆ yx) ≡ z ∆ yx ∆ yx        ⋄  (zyx ∆ yx) ≡ z ∆ ¯1               ⋄  (zyx ∆ yx) ≡ - z
    (zyx ∆ zy) ≡ z ∆ y ∆ x ∆ z ∆ y  ⋄  (zyx ∆ zy) ≡ - z ∆ z ∆ y ∆ x ∆ x  ⋄  (zyx ∆ zy) ≡ - y
    (zyx ∆ y)  ≡ z ∆ yx ∆ y         ⋄  (zyx ∆ y)  ≡ z ∆ -x               ⋄  (zyx ∆ y)  ≡ - zx
    (∆⍨ zyx)   ≡ z ∆ yx ∆ z ∆ yx    ⋄  (∆⍨ zyx)   ≡ yx ∆ yx              ⋄  (∆⍨zyx)    ≡ ¯1
    (zx ∆ yx)  ≡ z ∆ x ∆ y ∆ x      ⋄  (zx ∆ yx)  ≡ z ∆ -y               ⋄  (zx ∆ yx)  ≡ - zy

(the square of a bivector or a trivector is `¯1`, so `yx` and `zyx` are solutions to `¯1*0.5`, just like `0j1`!)

### Multiplication table

In the example above, we have calculated almost all possible products between unit blades. Let's create a table with all of them. We are going to define a function that, given a number of components, uses the bub to find the multiplication table.

In [33]:
⍝ eg
⍝ base unitary vectors for 1, 2 and 3 dimensions
b1 ← BB 2*1  ⍝ 1D bub: 2*1 elements (scalar, x)
b2 ← BB 2*2  ⍝ 2D bub: 2*2 elements (scalar, x, y, yx)
b3 ← BB 2*3  ⍝ 3D bub: 2*3 elements (scalar, x, y, yx, z, zx, zy, zyx)
'1D' '2D' '3D' Table b1 b2 b3

┌─────┬─────────────────┬─────────────────────────────────────────────────┐
│1D │2D │3D │
├─────┼─────────────────┼─────────────────────────────────────────────────┤
│┌─┬─┐│┌───┬───┬───┬───┐│┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐│
││0│1│││0 0│0 1│1 0│1 1│││0 0 0│0 0 1│0 1 0│0 1 1│1 0 0│1 0 1│1 1 0│1 1 1││
│└─┴─┘│└───┴───┴───┴───┘│└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘│
└─────┴─────────────────┴─────────────────────────────────────────────────┘

Parallel vectors, when multiplied, result in a scalar value. Therefore, if a 1-vector is in both factors, it will not be present in the product. For example, in:

    (zyx ∆ y)  ≡ - zx  ⋄  (zyx ∆ yx) ≡ - z  ⋄  (zx ∆ yx)  ≡ - zy  ⋄  ¯1 ≡ ∆⍨ yx
    
we see that when a vector (`x`, `y` or `z`) is found in both factors, it does not appear in the result. In other words, each vector will be present in the product if it is in one and only one of the factors. Therefore, to obtain the position of the product of two bub, we first perform a XOR (`≠`), and then convert it back to index value with `2⊥`.

In [34]:
PB ← 2⊥≠   ⍝ (P)osition of product of (B)ub
⍝ eg
'∘.≠' '∘.(2⊥≠)' Table (∘.≠⍨ b3) (∘.(2⊥≠)⍨ b3)
'1D' '2D' '3D' Table ∘.PB⍨¨ b1 b2 b3
'1D' '2D' '3D' Table (⌷∘id⊂)¨∘.PB⍨¨ b1 b2 b3

┌─────────────────────────────────────────────────┬───────────────┐
│∘.≠ │∘.(2⊥≠) │
├─────────────────────────────────────────────────┼───────────────┤
│┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐│0 1 2 3 4 5 6 7│
││0 0 0│0 0 1│0 1 0│0 1 1│1 0 0│1 0 1│1 1 0│1 1 1││1 0 3 2 5 4 7 6│
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│2 3 0 1 6 7 4 5│
││0 0 1│0 0 0│0 1 1│0 1 0│1 0 1│1 0 0│1 1 1│1 1 0││3 2 1 0 7 6 5 4│
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│4 5 6 7 0 1 2 3│
││0 1 0│0 1 1│0 0 0│0 0 1│1 1 0│1 1 1│1 0 0│1 0 1││5 4 7 6 1 0 3 2│
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│6 7 4 5 2 3 0 1│
││0 1 1│0 1 0│0 0 1│0 0 0│1 1 1│1 1 0│1 0 1│1 0 0││7 6 5 4 3 2 1 0│
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│ │
││1 0 0│1 0 1│1 1 0│1 1 1│0 0 0│0 0 1│0 1 0│0 1 1││ │
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│ │
││1 0 1│1 0 0│1 1 1│1 1 0│0 0 1│0 0 0│0 1 1│0 1 0││ │
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│ │
││1 1 0│1 1 1│1 0 0│1 0 1│0 1 0│0 1 1│0 0 0│0 0 1││ │
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│ │
││1 1 1│1 1 0│1 0 1│1 0 0│0 1 1│0 1 0│0 0 1│0 0 0││ │
│└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘│ │
└─────────────────────────────────────────────────┴───────────────┘

┌───┬───────┬───────────────┐
│1D │2D │3D │
├───┼───────┼───────────────┤
│0 1│0 1 2 3│0 1 2 3 4 5 6 7│
│1 0│1 0 3 2│1 0 3 2 5 4 7 6│
│ │2 3 0 1│2 3 0 1 6 7 4 5│
│ │3 2 1 0│3 2 1 0 7 6 5 4│
│ │ │4 5 6 7 0 1 2 3│
│ │ │5 4 7 6 1 0 3 2│
│ │ │6 7 4 5 2 3 0 1│
│ │ │7 6 5 4 3 2 1 0│
└───┴───────┴───────────────┘

┌─────┬─────────────┬─────────────────────────────────┐
│1D │2D │3D │
├─────┼─────────────┼─────────────────────────────────┤
│┌─┬─┐│┌──┬──┬──┬──┐│┌───┬───┬───┬───┬───┬───┬───┬───┐│
││1│x│││1 │x │y │yx│││1 │x │y │yx │z │zx │zy │zyx││
│├─┼─┤│├──┼──┼──┼──┤│├───┼───┼───┼───┼───┼───┼───┼───┤│
││x│1│││x │1 │yx│y │││x │1 │yx │y │zx │z │zyx│zy ││
│└─┴─┘│├──┼──┼──┼──┤│├───┼───┼───┼───┼───┼───┼───┼───┤│
│ ││y │yx│1 │x │││y │yx │1 │x │zy │zyx│z │zx ││
│ │├──┼──┼──┼──┤│├───┼───┼───┼───┼───┼───┼───┼───┤│
│ ││yx│y │x │1 │││yx │y │x │1 │zyx│zy │zx │z ││
│ │└──┴──┴──┴──┘│├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││z │zx │zy │zyx│1 │x │y │yx ││
│ │ │├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││zx │z │zyx│zy │x │1 │yx │y ││
│ │ │├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││zy │zyx│z │zx │y │yx │1 │x ││
│ │ │├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││zyx│zy │zx │z │yx │y │x │1 ││
│ │ │└───┴───┴───┴───┴───┴───┴───┴───┘│
└─────┴─────────────┴─────────────────────────────────┘

The n-dimensional table is just a subset of the (n+1)-dimensional table. We also see that the product is closed for a number of dimensions, but not for a number of components.

We also need to determine the sign resulting from the operation. By convention, we will assume that a product is positive when vectors of higher dimensions multiply vectors with of lower dimensions from left to right (ie. `0 0 0 1` is `1 0 ∆V 0 1`; the result of `0 1 ∆V 1 0` will be `0 0 0 ¯1`). And remember that we will multiply by `¯1` each time we switch positions. We need to count the number of vectors that are multiplying, by the right, vectors of higher dimensions, and multiply by `¯1` for each of them. Notice that we ignore the first component of the left argument (`1↓⊣`) and the last component of the right one (`¯1↓⊢`).

In [35]:
SB ← ¯1*(+\¯1↓⊢)+.×1↓⊣   ⍝ (S)ign of product of (B)ub
⍝ eg
l ← 0 1 1 0 1  ⋄  ln ← 'tzx'  ⋄  ll ← 1↓l                 ⍝ left argument
r ← 1 0 1 1 0  ⋄  rn ← 'uzy'  ⋄  rr ← +\¯1↓r              ⍝ right argument
s ← ' tzxuzy' '-txzuzy' ' txuy' '-tuxy' ' utxy' '-utyx'   ⍝ sequence
p ← ' ' 'zx ≡ -xz' 'uz ≡ -zu' 'xu ≡ -ux' 'tu ≡ -ut' 'xy ≡ -yx'
(ln,' ∆ ',rn) 'p' 'np' '¯1*p' Table∘(⍪¨) s p (⍳6) (1 ¯1 1 ¯1 1 ¯1)
h ← ('l←',ln) ('r←',rn) 'll←1↓l' 'rr←+\¯1↓r' 'll×rr' 'll+.×rr' '¯1*ll+.×rr'
h Table l r ll rr (ll×rr) (ll+.×rr) (¯1*ll+.×rr)
'1D' '2D' '3D' Table ∘.SB⍨¨ b1 b2 b3

┌─────────┬──────────┬──┬────┐
│tzx ∆ uzy│p │np│¯1*p│
├─────────┼──────────┼──┼────┤
│┌───────┐│┌────────┐│0 │ 1 │
││ tzxuzy│││ ││1 │¯1 │
│├───────┤│├────────┤│2 │ 1 │
││-txzuzy│││zx ≡ -xz││3 │¯1 │
│├───────┤│├────────┤│4 │ 1 │
││ txuy │││uz ≡ -zu││5 │¯1 │
│├───────┤│├────────┤│ │ │
││-tuxy │││xu ≡ -ux││ │ │
│├───────┤│├────────┤│ │ │
││ utxy │││tu ≡ -ut││ │ │
│├───────┤│├────────┤│ │ │
││-utyx │││xy ≡ -yx││ │ │
│└───────┘│└────────┘│ │ │
└─────────┴──────────┴──┴────┘

┌─────────┬─────────┬───────┬─────────┬───────┬───────┬──────────┐
│l←tzx │r←uzy │ll←1↓l │rr←+\¯1↓r│ll×rr │ll+.×rr│¯1*ll+.×rr│
├─────────┼─────────┼───────┼─────────┼───────┼───────┼──────────┤
│0 1 1 0 1│1 0 1 1 0│1 1 0 1│1 1 2 3 │1 1 0 3│5 │¯1 │
└─────────┴─────────┴───────┴─────────┴───────┴───────┴──────────┘

┌───┬─────────┬─────────────────────┐
│1D │2D │3D │
├───┼─────────┼─────────────────────┤
│1 1│1 1 1 1│1 1 1 1 1 1 1 1│
│1 1│1 1 ¯1 ¯1│1 1 ¯1 ¯1 ¯1 ¯1 1 1│
│ │1 1 1 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1│
│ │1 1 ¯1 ¯1│1 1 ¯1 ¯1 1 1 ¯1 ¯1│
│ │ │1 1 1 1 1 1 1 1│
│ │ │1 1 ¯1 ¯1 ¯1 ¯1 1 1│
│ │ │1 1 1 1 ¯1 ¯1 ¯1 ¯1│
│ │ │1 1 ¯1 ¯1 1 1 ¯1 ¯1│
└───┴─────────┴─────────────────────┘

In fact, we do not need to know the total number of permutations, only its parity, so we can instead use the `≠` function again:

In [36]:
SB ← ¯1*(≠\¯1↓⊢)≠.∧1↓⊣   ⍝ (S)ign of product of (B)ub
⍝ eg
Assert (∘.(¯1*(+\¯1↓⊢)+.×1↓⊣)⍨¨ ≡ ∘.SB⍨¨) b1 b2 b3

This is the function to calculate the multiplication table:

In [37]:
 BB  ← ↓∘⍉2⊥⍣¯1⍳           ⍝ (B)inary unitary (B)lades for multivectors of ⍵ components
 PB  ← 2⊥≠                 ⍝ (P)osition of product of (B)inary unitary blades
 SB  ← ¯1*(+\¯1↓⊢)+.×1↓⊣   ⍝ (S)ign of product of (B)inary unitary blades
 PS  ← (∘.PB,⍥⊂∘.SB)⍨∘BB   ⍝ (P)osition and (S)ign multiplication tables for multivectors of ⍵ components
 TD  ← PS(2*⊢)             ⍝ multiplication (T)ables for multivectors of ⍵ (D)imensions
 BN  ← ⊃((-1+⊣)↑¨⊢)/⍤PS    ⍝ (B)lades multiplication table for multivectors of ⍵ compo(N)ents
⊢BD  ← ⊃((-1+⊣)↑¨⊢)/⍤TD    ⍝ (B)lades multiplication table for multivectors of ⍵ (D)imensions
⍝ eg
'1D' '2D' Table BD¨1 2   ⍝ 1D and 2D blades tables
(⊂'3D')Table,⊂TD 3       ⍝ 3D positions and signs tables

┌┴─┐
 ⊢ ┌┴┐
 ⊃ ⍤
 ┌──────┴───────┐
 / ┌───┴───┐
 ┌─┘ ∘ ┌─┼─┐
 ┌───────┼─┐ ┌───┴───┐ 2 * ⊢
┌─┴─┐ ¨ ⊢ ⍨ ┌─┴──┐
- ┌─┼─┐ ┌─┘ ┌─┘ ∘ ┌─┼─┐
 1 + ⊣ ↑ ┌──────┼────┐ ┌┴┐ 2 ⍣ ⍳
 . ⍥ . ↓ ⍉ ┌┴┐
 ┌─┴─┐ ┌┴┐ ┌─┴──┐ ⊥ ¯1
 ∘ ┌─┼─┐ , ⊂ ∘ ┌──┼─────────┐
 2 ⊥ ≠ ¯1 * ┌───────┼────┐
 ┌─┴──┐ . ┌─┼─┐
 \ ┌──┼─┐ ┌┴┐ 1 ↓ ⊣
 ┌─┘ ¯1 ↓ ⊢ + ×
 +

┌─────────┬──────────────────────────────────┐
│1D │2D │
├─────────┼──────────────────────────────────┤
│┌───┬───┐│┌───────┬───────┬────────┬───────┐│
││1 │0 1│││1 │0 1 │0 0 1 │0 0 0 1││
│├───┼───┤│├───────┼───────┼────────┼───────┤│
││0 1│1 │││0 1 │1 │0 0 0 ¯1│0 0 ¯1 ││
│└───┴───┘│├───────┼───────┼────────┼───────┤│
│ ││0 0 1 │0 0 0 1│1 │0 1 ││
│ │├───────┼───────┼────────┼───────┤│
│ ││0 0 0 1│0 0 1 │0 ¯1 │¯1 ││
│ │└───────┴───────┴────────┴───────┘│
└─────────┴──────────────────────────────────┘

┌───────────────────────────────────────┐
│3D │
├───────────────────────────────────────┤
│┌───────────────┬─────────────────────┐│
││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1││
││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││2 3 0 1 6 7 4 5│1 1 1 1 ¯1 ¯1 ¯1 ¯1││
││3 2 1 0 7 6 5 4│1 1 ¯1 ¯1 1 1 ¯1 ¯1││
││4 5 6 7 0 1 2 3│1 1 1 1 1 1 1 1││
││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││6 7 4 5 2 3 0 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1││
││7 6 5 4 3 2 1 0│1 1 ¯1 ¯1 1 1 ¯1 ¯1││
│└───────────────┴─────────────────────┘│
└───────────────────────────────────────┘

## Geometric product of multivectors

In general, if we have two multivectors, we will apply the distributive property, so we need to calculate the outer product of its components, then multiply by the multiplication table, and add all the results. Let's see an example:

In [38]:
⍝ eg
'uv' Table u1 v1 ← ⊃¨¨u v                                               ⍝ multivectors of 3 components: 1, x, y
t ← u1(BN⌈⍥≢)v1                                                         ⍝ multiplication table
't' 'u∘.×v' 'u(t×∘.×)v'       Table t (u1∘.×v1) (u1(t×∘.×)v1)           ⍝ products
'↑,u(t×∘.×)v' '+⌿↑,u(t×∘.×)v' Table (↑,u1(t×∘.×)v1) (+⌿↑,u1(t×∘.×)v1)   ⍝ mix and reduce

┌──────────────┬─────────────────┐
│u │v │
├──────────────┼─────────────────┤
│4.273 0 0.6168│2.091 2.395 2.965│
└──────────────┴─────────────────┘

┌────────────────────────┬───────────────────┬────────────────────────────────┐
│t │u∘.×v │u(t×∘.×)v │
├────────────────────────┼───────────────────┼────────────────────────────────┤
│┌─────┬───────┬────────┐│8.937 10.24 12.67 │┌────────┬───────────┬─────────┐│
││1 │0 1 │0 0 1 ││0 0 0 ││8.937 │0 10.24 │0 0 12.67││
│├─────┼───────┼────────┤│1.29 1.477 1.829│├────────┼───────────┼─────────┤│
││0 1 │1 │0 0 0 ¯1││ ││0 0 │0 │0 0 0 0 ││
│├─────┼───────┼────────┤│ │├────────┼───────────┼─────────┤│
││0 0 1│0 0 0 1│1 ││ ││0 0 1.29│0 0 0 1.477│1.829 ││
│└─────┴───────┴────────┘│ │└────────┴───────────┴─────────┘│
└────────────────────────┴───────────────────┴────────────────────────────────┘

┌───────────────────────┬───────────────────────┐
│↑,u(t×∘.×)v │+⌿↑,u(t×∘.×)v │
├───────────────────────┼───────────────────────┤
│8.937 0 0 0 │10.77 10.24 13.96 1.477│
│0 10.24 0 0 │ │
│0 0 12.67 0 │ │
│0 0 0 0 │ │
│0 0 0 0 │ │
│0 0 0 0 │ │
│0 0 1.29 0 │ │
│0 0 0 1.477│ │
│1.829 0 0 0 │ │
└───────────────────────┴───────────────────────┘

A function to calculate the geometric product of two multivectors can therefore be defined as:

In [39]:
GP ← {t ← ⍺(BN⌈⍥≢)⍵ ⋄ +⌿↑,⍺(t×(⍴t)⊂¨⍤↑∘.×)⍵}   ⍝ geometric product
⍝ or as a tacit expression:
⊢ P ← +⌿∘↑∘,∘.×(⊂¨⍤↑⍨∘⍴×⊢)BN⍤⌈⍥≢
⍝ eg
'u' 'v' 'u P⍥V v' Table u v (u P⍥V v)
Assert (u P v) ≡ u GP v

┌──┴───┐
 ⊢ ┌────┴────┐
 ∘ ┌─────┼──────────┐
 ┌┴┐ . ┌─┼─┐ ⍥
 ∘ , ┌┴┐ ∘ × ⊢ ┌┴┐
 ┌┴┐ ∘ × ┌┴┐ ⍤ ≢
 ⌿ ↑ ⍨ ⍴ ┌┴─┐
┌─┘ ┌─┘ ┌┴┐ ⌈
+ ⍤ ⊃ ⍤
 ┌┴┐ ┌─────┴──────┐
 ¨ ↑ / ∘
 ┌─┘ ┌─┘ ┌───┴───┐
 ⊂ ┌───────┼─┐ ⍨ ┌─┴──┐
 ┌─┴─┐ ¨ ⊢ ┌─┘ ∘ ┌─┼─┐
 - ┌─┼─┐ ┌─┘ ┌──────┼────┐ ┌┴┐ 2 ⍣ ⍳
 1 + ⊣ ↑ . ⍥ . ↓ ⍉ ┌┴┐
 ┌─┴─┐ ┌┴┐ ┌─┴──┐ ⊥ ¯1
 ∘ ┌─┼─┐ , ⊂ ∘ ┌──┼─────────┐
 2 ⊥ ≠ ¯1 * ┌───────┼────┐
 ┌─┴──┐ . ┌─┼─┐
 \ ┌──┼─┐ ┌┴┐ 1 ↓ ⊣
 ┌─┘ ¯1 ↓ ⊢ + ×
 +

┌────────────────┬─────────────────────┬─────────────────────────────────────────┐
│u │v │u P⍥V v │
├────────────────┼─────────────────────┼─────────────────────────────────────────┤
│┌─────┬─┬──────┐│┌─────┬──────┬──────┐│┌──────┬─┬─┬────────┬─┬────────┬──────┬─┐│
││4.273│0│0.6168│││2.091│2.395 │2.965 │││10.77 │0│0│¯10.24 │0│¯11.38 │ 1.477│0││
││1.203│ │2.405 │││2.992│1.062 │1.84 │││ 8.026│0│0│ ¯1.278 │0│ 4.982 │ 2.554│0││
││1.929│ │0.9094│││2.048│0.2925│0.5153│││ 4.419│0│0│ ¯0.5644│0│ 0.8677│ 0.266│0││
││1.21 │ │4.783 │││1.754│2.754 │4.472 │││23.51 │0│0│ ¯3.332 │0│ 2.979 │13.17 │0││
││1.23 │ │2.414 │││1.887│2.398 │1.623 │││ 6.24 │0│0│ ¯2.951 │0│ 2.558 │ 5.79 │0││
│└─────┴─┴──────┘│└─────┴──────┴──────┘│└──────┴─┴─┴────────┴─┴────────┴──────┴─┘│
└────────────────┴─────────────────────┴─────────────────────────────────────────┘

Notice that we enclose the elements of the result of the outer product to allow the multiplication of nested multivectors. We are also going to define an operator `_GP_` that takes as left operand the multiplication function, and as right operand the multiplication table or a function to calculate it:

In [40]:
_GP_ ← {t ← ⍺(⍵⍵⍨⍨)⍵ ⋄ +⌿↑,⍺(t×∘⊂¨(⍴t)↑∘.(⍺⍺_Z))⍵}   ⍝ geometric product operator
_GP  ← _GP_(BN⌈⍥≢)                                   ⍝ geometric product operator (will compute table)
_GP2 ← _GP_(BD 2)                                    ⍝ geometric product 2D (will use precomputed table)
 GP  ← ×_GP  _V                                      ⍝ geometric product function (will compute table)
 GP2 ← ×_GP2 _V                                      ⍝ geometric product 2D (will use precomputed table)
⍝ eg
v1 ← (1 2) (3 4) ⋄ v2 ← (5 6) (7 8)   ⍝ 2D vectors
'v1' 'v2' '×_GP' '∘.×_GP' '+.×_GP' Table v1 v2 (v1(×_GP)⍥V v2) (v1(∘.×_GP)⍥V v2) (v1(+.×_GP)⍥V v2)
]RunTime 'v1   ×_GP v2' 'v1   ×_GP2 v2' -compare
]RunTime 'v1 ∘.×_GP v2' 'v1 ∘.×_GP2 v2' -compare

┌─────────┬─────────┬───────────────────┬────────────────────┬─────────┐
│v1 │v2 │×_GP │∘.×_GP │+.×_GP │
├─────────┼─────────┼───────────────────┼────────────────────┼─────────┤
│┌───┬───┐│┌───┬───┐│┌─────┬───┬───┬───┐│┌─────┬───┬───┬────┐│70 0 0 16│
││1 2│3 4│││5 6│7 8│││26 44│0 0│0 0│8 8│││26 30│0 0│0 0│8 10││ │
│└───┴───┘│└───┴───┘│└─────┴───┴───┴───┘││38 44│0 0│0 0│6 8││ │
│ │ │ │└─────┴───┴───┴────┘│ │
└─────────┴─────────┴───────────────────┴────────────────────┴─────────┘

v1 ×_GP v2 → 1.6E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* v1 ×_GP2 v2 → 1.3E¯5 | -22% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

v1 ∘.×_GP v2 → 1.7E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* v1 ∘.×_GP2 v2 → 1.5E¯5 | -14% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

Using a precomputed table suposes only a small improvement. We will work on the performance of the geometric product later.

## Properties of the geometric product

As it has been defined, the result of multiplying a vector by its inverse using the geometric product (`GP∘I⍨`) must result in the scalar `1`. Indeed:

In [41]:
Assert 1 MV¨ (GP∘I⍨)∘V¨ u v w

VALUE ERROR: Undefined name: MV
      Assert 1 MV¨(GP∘I⍨)∘V¨u v w
               ∧


This is not the only property that a geometric product must fulfill. In fact, we have already used several other properties such as associativity or the distributive property. Let's clearly define the properties of the geometric product:

In [42]:
]dinput
_GA ← {                                            ⍝ (G)eometric (A)lgebra
    _ ← Assert (a b ← ⍺) VS (u v w ← ⍵)            ⍝ vector space: a b scalars, u v w multivectors
    ∆ ← ⍺⍺                                         ⍝ geometric product function
    b ← V¨BV⌈/D¨⍵                                  ⍝ unitary vectors (base)
                                                   ⍝ properties:
    _ ← Assert (a ∆ v)          MV a × v           ⍝   left-product by scalar is scalar product
    _ ← Assert (v ∆ a)          MV v × a           ⍝   right-product by scalar is scalar product
    _ ← Assert (u ∆ v ∆ w)      MV (u ∆ v) ∆ w     ⍝   associative product
    _ ← Assert ((u +_V v) ∆ w)  MV u +_V⍥(∆∘w) v   ⍝   distributive by right
    _ ← Assert (u ∆ (v +_V w))  MV v +_V⍥(u∘∆) w   ⍝   distributive by left
    _ ← Assert (1⍴⍨≢b)          MV¨∆⍨¨ b           ⍝   unitary vectors square to 1
    _ ← Assert                     ∆/1,b           ⍝   the product of orthonormal vectors is not zero
    1
}

When we have a vector space with a product that follows these properties, we say we have a *geometric algebra*. The operator `_GA` is analogous to `VS`, but takes as operand a geometric product function. If we try the functions we already have:

In [43]:
a b (GP  _GA)V¨ u v w   ⍝ geometric algebra with geometric product ∆ (u v w are vectors)
a b (GP2 _GA)   u v w   ⍝ geometric algebra with geometric product ∆2 (u v w are multivectors)

VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧
VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧


The `GP` and `GP2` functions are geometric products.

Inside `_GA`, we are making sure that the geometric product follows the properties that we have defined. The square of unitary vectors must be one (`(1⍴⍨≢b)MV ∆⍨¨b`). Therefore, as long as the other properties hold, unitary vectors will be their own inverses, and the square of vectors the square of their magnitudes. Moreover, this property is the reason the product of orthonormal vectors is anti-commutative. We also impose that the product of orthonormal vectors cannot be zero. This is checked with `∆/1,b` (`b` is prefixed by a `1` to explicitly allow 0-dimensional spaces). Notice that, due to the scalar property (left and right products by scalar are scalar products), this single condition guarantees that all possible products are not zero. :

    0 ≢ zyx  ⋄  0 ≢ z ∆ y ∆ x  ⋄  0 ≢ z ∆ y  ⋄  0 ≢ y ∆ x
                0 ≢-z ∆ x ∆ y  ⋄  0 ≢ z ∆ x

## Improving the performance of the geometric product

The performance of our geometric product can be improved. We can do much better using the `V` function defined before instead of mix and reduce. As we have defined it, `V` can be used to add blades into a multivector. For example, we can use it to calculate the unit blades product table from the tables of positions and signs:

In [44]:
BT ← ⊃V¨/     ⍝ (B)lades from position and sign (T)ables
BN  ← BT PS   ⍝ (B)lades multiplication table for (N)umber of components ⍵
BD  ← BT TD   ⍝ (B)lades multiplication table for ⍵ (D)imensions
⍝ eg
'1D' '2D' Table BD¨1 2   ⍝ same result as before

┌─────────┬──────────────────────────────────┐
│1D │2D │
├─────────┼──────────────────────────────────┤
│┌───┬───┐│┌───────┬───────┬────────┬───────┐│
││1 │0 1│││1 │0 1 │0 0 1 │0 0 0 1││
│├───┼───┤│├───────┼───────┼────────┼───────┤│
││0 1│1 │││0 1 │1 │0 0 0 ¯1│0 0 ¯1 ││
│└───┴───┘│├───────┼───────┼────────┼───────┤│
│ ││0 0 1 │0 0 0 1│1 │0 1 ││
│ │├───────┼───────┼────────┼───────┤│
│ ││0 0 0 1│0 0 1 │0 ¯1 │¯1 ││
│ │└───────┴───────┴────────┴───────┘│
└─────────┴──────────────────────────────────┘

And we can also use it in the definition of the geometric product. This is our new version:

In [45]:
Vo ← {⍺←2*⍳≢⍵ ⋄ v←0⍴⍨1+⌈/,⍺ ⋄ v[⍺]+←(⍴⍺)↑⍵ ⋄ v}   ⍝ multi(V)ector with blades ⍵ at ⍺ (1-vector by default)
V ← {⍺←2*⍳≢⍵ ⋄ v←0⍴⍨1+⌈/,a←(r←⍺⌊⍥⍴⍵)↑⍺ ⋄ v[a]+←r↑⍵ ⋄ v}   ⍝ multi(V)ector with blades ⍵ at ⍺ (1-vector by default)
⍝V ← {⍺←2*⍳≢⍵ ⋄ a w←⍺⍵↑⍨¨⊂⍺⌊⍥⍴⍵ ⋄ v←0⍴⍨1+⌈/,a ⋄ v[a]+←w ⋄ v}   ⍝ multi(V)ector with blades ⍵ at ⍺ (1-vector by default)
V ← {⍺←2*⍳≢⍵ ⋄ a←⍺↑⍨r←⍺⌊⍥⍴⍵ ⋄ v←0⍴⍨1+⌈/,a ⋄ v[a]+←r↑⍵ ⋄ v}   ⍝ multi(V)ector with blades ⍵ at ⍺ (1-vector by default)
_Z  ← {⍺←⊢ ⋄ 0∊⍺⍵:0 ⋄ ⍺(⍺⍺)⍵}                       ⍝ result (Z)ero for zero argument
_Z  ← {(0≡⍵)∨0≡⍺:0 ⋄ ⍺(⍺⍺)⍵}                       ⍝ result (Z)ero for zero argument
⍝_Z  ← {⍺←⊢ ⋄ (0∊⍺⍵)⊃(⍺(⍺⍺)⍵) 0}                       ⍝ result (Z)ero for zero argument
⍝_Z  ← {{0}⍣(0∊⍺⍵)⊢⍺(⍺⍺)⍵}                       ⍝ result (Z)ero for zero argument
⍝ GEOMETRIC PRODUCT
_∆_ ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ r←(≢¨⍺⍵)⌊⍴p ⋄ (r↑p)V(r↑s)×r↑⍺∘.(⍺⍺_Z)⍥,⍵}        ⍝ ⍺⍺ product, ⍵⍵ tables or tables func
⍝_∆_ ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ r←(≢¨⍺⍵)⌊⍴p ⋄ p V(r↑s)×r↑⍺∘.(⍺⍺_Z)⍥,⍵}        ⍝ ⍺⍺ product, ⍵⍵ tables or tables func
_∆  ← _∆_(PS⌈⍥≢)                                                       ⍝ ⍺⍺ product (table will be computed)
 ∆  ← ×_∆                                                              ⍝ geometric product
 ∆2 ← ×_∆_(TD 2)                                                       ⍝ 2D geometric product
⍝ eg
'v1' 'v2' '×_∆' '∘.×_∆' '+.×_∆' Table v1 v2 (v1(×_∆)⍥V v2) (v1(∘.×_∆)⍥V v2) (v1(+.×_∆)⍥V v2)
Assert a b (∆  _GA)V¨ u v w   ⍝ ∆ is a geometric product
Assert a b (∆2 _GA)   u v w   ⍝ ∆2 is a geometric product
Assert (u(∆ MV ∆2)v)
'u' 'v' 'u ∆ v'  Table (V u) (V v) (u ∆⍥V v)

┌─────────┬─────────┬───────────────┬────────────────┬─────────┐
│v1 │v2 │×_∆ │∘.×_∆ │+.×_∆ │
├─────────┼─────────┼───────────────┼────────────────┼─────────┤
│┌───┬───┐│┌───┬───┐│┌─────┬─┬─┬───┐│┌─────┬─┬─┬────┐│70 0 0 16│
││1 2│3 4│││5 6│7 8│││26 44│0│0│8 8│││26 30│0│0│8 10││ │
│└───┴───┘│└───┴───┘│└─────┴─┴─┴───┘││38 44│ │ │6 8││ │
│ │ │ │└─────┴─┴─┴────┘│ │
└─────────┴─────────┴───────────────┴────────────────┴─────────┘

VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧
VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧
VALUE ERROR: Undefined name: MV
      Assert(u(∆ MV ∆2)v)
                 ∧


┌────────────────────┬─────────────────────────┬─────────────────────────────────────────┐
│u │v │u ∆ v │
├────────────────────┼─────────────────────────┼─────────────────────────────────────────┤
│┌─┬─────┬─┬─┬──────┐│┌─┬─────┬──────┬─┬──────┐│┌──────┬─┬─┬────────┬─┬────────┬──────┬─┐│
││0│4.273│0│0│0.6168│││0│2.091│2.395 │0│2.965 │││10.77 │0│0│¯10.24 │0│¯11.38 │ 1.477│0││
││ │1.203│ │ │2.405 │││ │2.992│1.062 │ │1.84 │││ 8.026│ │ │ ¯1.278 │ │ 4.982 │ 2.554│ ││
││ │1.929│ │ │0.9094│││ │2.048│0.2925│ │0.5153│││ 4.419│ │ │ ¯0.5644│ │ 0.8677│ 0.266│ ││
││ │1.21 │ │ │4.783 │││ │1.754│2.754 │ │4.472 │││23.51 │ │ │ ¯3.332 │ │ 2.979 │13.17 │ ││
││ │1.23 │ │ │2.414 │││ │1.887│2.398 │ │1.623 │││ 6.24 │ │ │ ¯2.951 │ │ 2.558 │ 5.79 │ ││
│└─┴─────┴─┴─┴──────┘│└─┴─────┴──────┴─┴──────┘│└──────┴─┴─┴────────┴─┴────────┴──────┴─┘│
└────────────────────┴─────────────────────────┴─────────────────────────────────────────┘

We have already checked (with the help of `_GA`) that `∆` is a geometric product. Let's check the performance and memory requirements:

In [46]:
us vs ← V¨RC∘1e4¨3 4   ⍝ large vectors (3 and 4 components of 10k rows each)
_∆y_ ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ r←(≢¨⍺⍵)⌊⍴p ⋄ (r↑p)Vo(r↑s)×r↑⍺∘.(⍺⍺)⍥,⍵}    ⍝ ⍺⍺ product, ⍵⍵ tables or tables func
_∆y ← _∆y_(PS⌈⍥≢)
y3 ← ×_∆y_(TD 3)
y2 ← ×_∆y_(TD 2)
'v1' 'v2' '×_∆' '∘.×_∆' '+.×_∆' Table v1 v2 (v1(×_∆y)⍥V v2) (v1(∘.×_∆y)⍥V v2) (v1(+.×_∆y)⍥V v2)
GP3 ← ×_GP_(BD 3)
GP2 ← ×_GP_(BD 2)
∆3 ← ×_∆_(TD 3)
]RunTime     'u  ∘.×_∆⍥V v ' 'u  ∘.×_GP⍥V v ' 'u  ∘.×_Z _∆y⍥V v ' -compare
]RunTime     'u  ∆3⍥V v ' 'u  GP3⍥V v ' 'u  y3⍥V v ' -compare
]RunTime     'us ∆2   vs' 'us GP2   vs' 'us y2   vs' -compare
]SpaceNeeded 'u  ∆3⍥V v ' 'u  GP3⍥V v ' 'u  y3⍥V v '
]SpaceNeeded 'us ∆2   vs' 'us GP2   vs' 'us y2   vs'

┌─────────┬─────────┬───────────────────┬────────────────────┬─────────┐
│v1 │v2 │×_∆ │∘.×_∆ │+.×_∆ │
├─────────┼─────────┼───────────────────┼────────────────────┼─────────┤
│┌───┬───┐│┌───┬───┐│┌─────┬───┬───┬───┐│┌─────┬───┬───┬────┐│70 0 0 16│
││1 2│3 4│││5 6│7 8│││26 44│0 0│0 0│8 8│││26 30│0 0│0 0│8 10││ │
│└───┴───┘│└───┴───┘│└─────┴───┴───┴───┘││38 44│ │ │6 8││ │
│ │ │ │└─────┴───┴───┴────┘│ │
└─────────┴─────────┴───────────────────┴────────────────────┴─────────┘

u ∘.×_∆⍥V v → 5.5E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u ∘.×_GP⍥V v → 6.7E¯5 | +22% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 u ∘.×_Z _∆y⍥V v → 5.8E¯5 | +5% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

u ∆3⍥V v → 2.2E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u GP3⍥V v → 4.7E¯5 | +116% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u y3⍥V v → 1.3E¯5 | -42% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

us ∆2 vs → 2.6E¯4 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* us GP2 vs → 2.9E¯4 | +11% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* us y2 vs → 2.6E¯4 | -3% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌───────────┬────┐
│u ∆3⍥V v │2868│
├───────────┼────┤
│u GP3⍥V v │9140│
├───────────┼────┤
│u y3⍥V v │3308│
└───────────┴────┘

┌───────────┬──────┐
│us ∆2 vs │961236│
├───────────┼──────┤
│us GP2 vs│961516│
├───────────┼──────┤
│us y2 vs │990756│
└───────────┴──────┘

That's quite faster for small vectors, and much faster for the larger ones. It even needs less space. Of course, when a precomputed multiplication table is not used, it takes a bit longer (and more space), but much less than using `GP`:

In [47]:
]RunTime     'u  ∆3⍥V v ' 'u  ∆ ⍥V v ' 'u  GP⍥V v ' -compare
]RunTime     'us ∆2   vs' 'us ∆    vs' 'us GP   vs' -compare
]SpaceNeeded 'u  ∆3⍥V v ' 'u  ∆ ⍥V v ' 'u  GP⍥V v '
]SpaceNeeded 'us ∆2   vs' 'us ∆    vs' 'us GP   vs'

u ∆3⍥V v → 2.3E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 u ∆ ⍥V v → 5.9E¯5 | +158% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u GP⍥V v → 9.7E¯5 | +326% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

us ∆2 vs → 1.0E¯4 | 0% 
* us ∆ vs → 2.5E¯4 | +149% ⎕ 
* us GP vs → 1.0E¯2 | +10064% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌──────────┬────┐
│u ∆3⍥V v │2868│
├──────────┼────┤
│u ∆ ⍥V v │2940│
├──────────┼────┤
│u GP⍥V v │6348│
└──────────┴────┘

┌──────────┬───────┐
│us ∆2 vs│961236 │
├──────────┼───────┤
│us ∆ vs│1442908│
├──────────┼───────┤
│us GP vs│1639300│
└──────────┴───────┘

In [48]:
∆y ← ×_∆y_(PS⌈⍥≢)
]RunTime     'u  ∆3⍥V v ' 'u  ∆ ⍥V v ' 'u  ∆y⍥V v ' 'u  GP⍥V v ' -compare
]RunTime     'us ∆2   vs' 'us ∆    vs' 'us ∆y   vs' 'us GP   vs' -compare
]SpaceNeeded 'u  ∆3⍥V v ' 'u  ∆y⍥V v ' 'u  ∆ ⍥V v ' 'u  GP⍥V v '
]SpaceNeeded 'us ∆2   vs' 'us ∆y   vs' 'us ∆    vs' 'us GP   vs'

u ∆3⍥V v → 3.9E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 u ∆ ⍥V v → 1.1E¯4 | +172% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u ∆y⍥V v → 6.3E¯5 | +61% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u GP⍥V v → 1.2E¯4 | +217% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

us ∆2 vs → 1.1E¯4 | 0% 
* us ∆ vs → 2.6E¯4 | +138% ⎕ 
* us ∆y vs → 2.9E¯4 | +174% ⎕ 
* us GP vs → 1.1E¯2 | +10232% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌──────────┬────┐
│u ∆3⍥V v │2868│
├──────────┼────┤
│u ∆y⍥V v │3380│
├──────────┼────┤
│u ∆ ⍥V v │2940│
├──────────┼────┤
│u GP⍥V v │6348│
└──────────┴────┘

┌──────────┬───────┐
│us ∆2 vs│961236 │
├──────────┼───────┤
│us ∆y vs│1479756│
├──────────┼───────┤
│us ∆ vs│1442908│
├──────────┼───────┤
│us GP vs│1639300│
└──────────┴───────┘

## Products of vectors and blades

Finally, we are going to define some additional functions to make it easier to multiply vectors, just for convenience:

In [49]:
∆V ← ∆⍥V   ⍝ geometric product of vectors
∆v ← ∆∘V   ⍝ geometric product of multivector and vector
⍝ eg: 3D unitary blades: scalar, vectors, bivectors, trivector
x y z        ← 1 (0 1) (0 0 1)                   ⍝ unitary vectors
yx zx zy zyx ← (y z z ∆V¨ x x y),⊂z ∆v⍨ y ∆V x   ⍝ unitary bivectors and trivector
' ' 'x' 'y' 'z' 'yx' 'zx' 'zy' 'zyx' Table 1,(V¨x y z),yx zx zy zyx

┌─┬───┬─────┬─────────┬───────┬───────────┬─────────────┬───────────────┐
│ │x │y │z │yx │zx │zy │zyx │
├─┼───┼─────┼─────────┼───────┼───────────┼─────────────┼───────────────┤
│1│0 1│0 0 1│0 0 0 0 1│0 0 0 1│0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 1│
└─┴───┴─────┴─────────┴───────┴───────────┴─────────────┴───────────────┘

We can use these new functions to study what is the result of multiplying different blades. For instance, we can observe that the product of perpendicular vectors or penpendicular bivectors is anticommutative, but the product of bivectors and orthogonal vectors is commutative:

In [50]:
⍝ anticommutative product of vector and orthogonal vector
Assert (z ∆V y) MV - y ∆V z
Assert (y ∆V x) MV - x ∆V y
Assert (x ∆V z) MV - z ∆V x
⍝ anticommutative product of bivector and orthogonal bivector
Assert (yx ∆ zy) MV - zy ∆ yx
Assert (zy ∆ zx) MV - zx ∆ zy
Assert (zx ∆ yx) MV - yx ∆ zx
⍝ commutative product of bivector and orthogonal vector
Assert ((V z) ∆ y ∆V x) ≡ z ∆v⍨ y ∆V x
Assert ((V y) ∆ z ∆V x) ≡ y ∆v⍨ z ∆V x
Assert ((V x) ∆ z ∆V y) ≡ x ∆v⍨ z ∆V y

VALUE ERROR: Undefined name: MV
      Assert(z ∆V y)MV-y ∆V z
                    ∧
VALUE ERROR: Undefined name: MV
      Assert(y ∆V x)MV-x ∆V y
                    ∧
VALUE ERROR: Undefined name: MV
      Assert(x ∆V z)MV-z ∆V x
                    ∧
VALUE ERROR: Undefined name: MV
      Assert(yx ∆ zy)MV-zy ∆ yx
                     ∧
VALUE ERROR: Undefined name: MV
      Assert(zy ∆ zx)MV-zx ∆ zy
                     ∧
VALUE ERROR: Undefined name: MV
      Assert(zx ∆ yx)MV-yx ∆ zx
                     ∧


# Signed geometric algebras

## Non-proper vectors

Until now, all the vectors that we have considered had the property `(∆⍨ v) ≡ +.×⍨ v`. Consequently, all vectors, when squared, resulted in a positive number (and unitary vectors, in particular, square to `1`). We will call the vectors that square to positive scalars *vectors with positive metric*, or 0<vectors. However, it is possible to find vectors with different square values. They will be called *non-proper vectors*.

### Vectors of negative metric

For example, let's consider a 3D space and calculate the square value of each of the unit blades.

In [51]:
gr ← G¨bu                ⍝ grades
sq ← ∆⍨¨bu               ⍝ squares
vs ← (a b VS u v,⊂)¨bu   ⍝ is a vector?
'unitary blades' 'bub' 'name' 'grade' '∆⍨' 'vector?' Table∘(⍪¨) bu bub id gr sq vs

┌─────────────────┬───────┬─────┬─────┬──────────────────┬───────┐
│unitary blades │bub │name │grade│∆⍨ │vector?│
├─────────────────┼───────┼─────┼─────┼──────────────────┼───────┤
│┌───────────────┐│┌─────┐│┌───┐│0 │┌────────────────┐│1 │
││1 │││0 0 0│││1 ││1 ││1 ││1 │
│├───────────────┤│├─────┤│├───┤│1 │├────────────────┤│1 │
││0 1 │││0 0 1│││x ││2 ││1 0 ││1 │
│├───────────────┤│├─────┤│├───┤│1 │├────────────────┤│1 │
││0 0 1 │││0 1 0│││y ││2 ││1 0 0 0 ││1 │
│├───────────────┤│├─────┤│├───┤│2 │├────────────────┤│1 │
││0 0 0 1 │││0 1 1│││yx ││3 ││¯1 0 0 0 ││1 │
│├───────────────┤│├─────┤│├───┤│ │├────────────────┤│ │
││0 0 0 0 1 │││1 0 0│││z ││ ││1 0 0 0 0 0 0 0 ││ │
│├───────────────┤│├─────┤│├───┤│ │├────────────────┤│ │
││0 0 0 0 0 1 │││1 0 1│││zx ││ ││¯1 0 0 0 0 0 0 0││ │
│├───────────────┤│├─────┤│├───┤│ │├────────────────┤│ │
││0 0 0 0 0 0 1 │││1 1 0│││zy ││ ││¯1 0 0 0 0 0 0 0││ │
│├───────────────┤│├─────┤│├───┤│ │├────────────────┤│ │
││0 0 0 0 0 0 0 1│││1 1 1│││zyx││ ││¯1 0 0 0 0 0 0 0││ │
│└───────────────┘│└─────┘│└───┘│ │└────────────────┘│ │
└─────────────────┴───────┴─────┴─────┴──────────────────┴───────┘

As we saw above, some unitary blades (the bivectors and trivectors) have negative squares. Nevertheless, they are vectors according to our definition of vector spaces (as checked by `a b VS u v,⊂`). They will be called *vectors with negative metric*, or 0>vectors. The 0>vectors also fulfill the anticommutative property for orthogonal vectors of the geometric product. However they do not square to `1`, but to `¯1`.

In [52]:
b3 ← yx zx zy   ⍝ bivectors in 3D
' ' '∆⍨' '∆∘(1∘⌽)⍨' '∆⍨∘(1∘⌽)⍨' '∆⍨≡-⍤∆' Table∘(⍪¨) b3 (∆¨⍨b3) (∆¨∘(1∘⌽)⍨b3) (∆⍨¨∘(1∘⌽)⍨b3) ((∆⍨≡-⍤∆)¨∘(1∘⌽)⍨b3)
a b (∆ _GA) b3

┌───────────────┬──────────────────┬─────────────────┬──────────────────┬──────┐
│ │∆⍨ │∆∘(1∘⌽)⍨ │∆⍨∘(1∘⌽)⍨ │∆⍨≡-⍤∆│
├───────────────┼──────────────────┼─────────────────┼──────────────────┼──────┤
│┌─────────────┐│┌────────────────┐│┌───────────────┐│┌────────────────┐│1 │
││0 0 0 1 │││¯1 0 0 0 │││0 0 0 0 0 0 1 0│││0 0 0 0 0 0 ¯1 0││1 │
│├─────────────┤│├────────────────┤│├───────────────┤│├────────────────┤│1 │
││0 0 0 0 0 1 │││¯1 0 0 0 0 0 0 0│││0 0 0 1 0 0 0 0│││0 0 0 ¯1 0 0 0 0││ │
│├─────────────┤│├────────────────┤│├───────────────┤│├────────────────┤│ │
││0 0 0 0 0 0 1│││¯1 0 0 0 0 0 0 0│││0 0 0 0 0 1 0 0│││0 0 0 0 0 ¯1 0 0││ │
│└─────────────┘│└────────────────┘│└───────────────┘│└────────────────┘│ │
└───────────────┴──────────────────┴─────────────────┴──────────────────┴──────┘

VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧


Then, we could define a vector space, in which the base is formed by `1` and the bivectors `yx`, `zx` and `zy`, for example (this space would actually be the *[quaternion](https://en.wikipedia.org/wiki/Quaternion_algebra)* space). We will use our geometric product `∆` to calculate the corresponding multiplication table, and the functions `VQ` and `QV` to change of basis between quaternions and 3D multivectors:

In [53]:
VQ ← Z∘C 1 ¯2 1 0 1 1\4∘X   ⍝ multivector from quaternion
QV ← Z∘C (⊂0 3 5 6)⌷7∘X     ⍝ quaternion from multivector
⍝ eg
'q' 'VQ q' 'v' 'QV v' Table (1+⍳4) (VQ 1+⍳4) (1+⍳7) (QV 1+⍳7)   ⍝ change of basis
bq ← 1 yx zx zy                                                 ⍝ quaternions base
'basis' '∘.∆⍨' Table (⍪bq) (∘.∆⍨bq)                             ⍝ quaternions product table
'basis' 'QV¨∘.∆⍨' Table (⍪BV 4) (tq ← QV¨∘.∆⍨bq)                ⍝ change of basis

┌───────┬─────────────┬─────────────┬───────┐
│q │VQ q │v │QV v │
├───────┼─────────────┼─────────────┼───────┤
│1 2 3 4│1 0 0 2 0 3 4│1 2 3 4 5 6 7│1 4 6 7│
└───────┴─────────────┴─────────────┴───────┘

┌───────────────┬──────────────────────────────────────────────────────────────────┐
│basis │∘.∆⍨ │
├───────────────┼──────────────────────────────────────────────────────────────────┤
│┌─────────────┐│┌─────────────┬────────────────┬────────────────┬────────────────┐│
││1 │││1 │0 0 0 1 │0 0 0 0 0 1 │0 0 0 0 0 0 1 ││
│├─────────────┤│├─────────────┼────────────────┼────────────────┼────────────────┤│
││0 0 0 1 │││0 0 0 1 │¯1 0 0 0 │0 0 0 0 0 0 1 0 │0 0 0 0 0 ¯1 0 0││
│├─────────────┤│├─────────────┼────────────────┼────────────────┼────────────────┤│
││0 0 0 0 0 1 │││0 0 0 0 0 1 │0 0 0 0 0 0 ¯1 0│¯1 0 0 0 0 0 0 0│0 0 0 1 0 0 0 0 ││
│├─────────────┤│├─────────────┼────────────────┼────────────────┼────────────────┤│
││0 0 0 0 0 0 1│││0 0 0 0 0 0 1│0 0 0 0 0 1 0 0 │0 0 0 ¯1 0 0 0 0│¯1 0 0 0 0 0 0 0││
│└─────────────┘│└─────────────┴────────────────┴────────────────┴────────────────┘│
└───────────────┴──────────────────────────────────────────────────────────────────┘

┌─────────┬──────────────────────────────────┐
│basis │QV¨∘.∆⍨ │
├─────────┼──────────────────────────────────┤
│┌───────┐│┌───────┬────────┬───────┬───────┐│
││1 │││1 │0 1 │0 0 1 │0 0 0 1││
│├───────┤│├───────┼────────┼───────┼───────┤│
││0 1 │││0 1 │¯1 │0 0 0 1│0 0 ¯1 ││
│├───────┤│├───────┼────────┼───────┼───────┤│
││0 0 1 │││0 0 1 │0 0 0 ¯1│¯1 │0 1 ││
│├───────┤│├───────┼────────┼───────┼───────┤│
││0 0 0 1│││0 0 0 1│0 0 1 │0 ¯1 │¯1 ││
│└───────┘│└───────┴────────┴───────┴───────┘│
└─────────┴──────────────────────────────────┘

Having a multiplication table for basis blades, we can use `_GP_` to get a geometric product fuction. Or we could use the 3D multiplication under a change of basis:

In [54]:
Pq ← ×_GP_ tq _V   ⍝ quaternion product using quaternion table
∆Q ← QV P3⍥VQ      ⍝ 3D geometric product under change of basis
'u' 'v' 'VQ u' 'VQ v' Table u (v,5) (VQ u) (VQ v,5)
'u P3⍥VQ v' 'u Pq v' Table (u P3⍥VQ v,5) (u Pq v,5)
]RunTime 'u ∆Q v,5' 'u Pq v,5' -compare

VALUE ERROR: Undefined name: P3
      ∆Q←QV P3⍥VQ      ⍝ 3D geometric product under change of basis
            ∧


┌────────────────┬───────────────────────┬──────────────────────┬─────────────────────────────┐
│u │v │VQ u │VQ v │
├────────────────┼───────────────────────┼──────────────────────┼─────────────────────────────┤
│┌─────┬─┬──────┐│┌─────┬──────┬──────┬─┐│┌─────┬─┬─┬─┬─┬──────┐│┌─────┬─┬─┬──────┬─┬──────┬─┐│
││4.273│0│0.6168│││2.091│2.395 │2.965 │5│││4.273│0│0│0│0│0.6168│││2.091│0│0│2.395 │0│2.965 │5││
││1.203│ │2.405 │││2.992│1.062 │1.84 │ │││1.203│ │ │ │ │2.405 │││2.992│ │ │1.062 │ │1.84 │ ││
││1.929│ │0.9094│││2.048│0.2925│0.5153│ │││1.929│ │ │ │ │0.9094│││2.048│ │ │0.2925│ │0.5153│ ││
││1.21 │ │4.783 │││1.754│2.754 │4.472 │ │││1.21 │ │ │ │ │4.783 │││1.754│ │ │2.754 │ │4.472 │ ││
││1.23 │ │2.414 │││1.887│2.398 │1.623 │ │││1.23 │ │ │ │ │2.414 │││1.887│ │ │2.398 │ │1.623 │ ││
│└─────┴─┴──────┘│└─────┴──────┴──────┴─┘│└─────┴─┴─┴─┴─┴──────┘│└─────┴─┴─┴──────┴─┴──────┴─┘│
└────────────────┴───────────────────────┴──────────────────────┴─────────────────────────────┘

VALUE ERROR: Undefined name: P3
      'u P3⍥VQ v' 'u Pq v'Table(u P3⍥VQ v,5)(u Pq v,5)
                                  ∧


* Command Execution Failed: VALUE ERROR: Undefined name: ∆Q

However:

In [55]:
a b (Pq _GA) u v w   ⍝ ERROR: unit vectors must square to 1

VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧


But that is only because our definition specifically required that vectors squared to `1`. Our definition of vector inverses, which we came up with intuitively deducing what should be the inverse of a velocity, was a bit shortsighted. It worked well with velocities, but vectors can represent anything as long as the properties defined in `VS` are fulfilled. For example, a vector could represent a rotation, and in that case the inverse should be a rotation in the opposite direction (not a rotation of inverse magnitude). More generally, any blade of a multivector or any linear combination of blades will fulfill the vector space properties and could be part of the base of a vector space. We will fix this using signed geometric algebras.

### Null vectors

There is another kind of vectors: those ones for which the square is zero. They will be called *null vectors*, or 0=vectors. Null vectors can be obtained adding a trivector and a vector orthogonal to the trivector. For example:

In [56]:
t ← 0,zyx   ⍝ vector orthogonal to zyx
n ← zyx,1   ⍝ null vector
'zyx' 't' 'zyx +.×_V t' 'n ← zyx + t' '∆⍨ n' Table zyx t (zyx+.×_V t) n (∆⍨n)
Assert 0 ≡ zyx +.×_V t   ⍝ zyx and t are perpendicular (dot product zero)
Assert 0 ≡ ∆⍨n           ⍝ the square of zyx+t is zero
a b (∆ _GA) u v n

┌───────────────┬─────────────────┬───────────┬─────────────────┬───────────────────────────────┐
│zyx │t │zyx +.×_V t│n ← zyx + t │∆⍨ n │
├───────────────┼─────────────────┼───────────┼─────────────────┼───────────────────────────────┤
│0 0 0 0 0 0 0 1│0 0 0 0 0 0 0 0 1│0 │0 0 0 0 0 0 0 1 1│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
└───────────────┴─────────────────┴───────────┴─────────────────┴───────────────────────────────┘

assertion failure
      Assert 0≡∆⍨n           ⍝ the square of zyx+t is zero
      ∧
VALUE ERROR: Undefined name: MV
_GA[5] _←Assert(a ∆ v)MV a×v           ⍝   left-product by scalar is scalar product
                      ∧


We can also use null vectors as the basis of a vector space and define a product table. However, it is starting to get complicated having to change into a basis of 0<vectors. For example, for an algebra with three 0<vectors and one 0=vector, we would need `2` dimensions for the 0<vectors and `3+1` dimensions for the 0=vector. To represent the trivector we will need `2*2+3+1` components, so 64 components multivectors. However, if we directly used a basis with 0=vectors, we would only need 8 components (`2*3`). Signed geometric algebras allow us to define geometric algebra with a basis that contains non-proper vectors.

## Signature

We will specify what kind of vectors we are using as the basis of a geometric algebra with the *signature* of the algebra. The signature is a list of up to three numbers: the first one specifies the number of 0<vectors, the second one the number of 0>vectors, and the third one the number of 0=vectors. If there are missing items, it is assumed that there are zero vectors of that kind. A geometric algebra with a specific signature will be called a *signed geometric algebra*.

For example, the algebra of quaternions will correspond to a geometric algebra with signature `0 2` (two of the unit quaternions will be vectors that square to `¯1` and the third one the bivector obtained multiplying them). The algebra with a 0<vector and a 0=vector would have the signature `1 0 1`. Other examples of signed geometric algebras will be presented later.

We will define a `_GA_` operator analogous to `_GA` but which can also take a signature (as right operand). It makes use of the `QS` function, which will take as argument a signutare and return the metric of the space, ie. the value of the squared vectors.

In [57]:
QS ← ⊃1 ¯1 0,.⍴⍨(⊢↑⍨3⌈≢)   ⍝ s(Q)uared vectors for (S)ignature ⍵
⍝ eg
ss ← 3 (0 2) (3 0 1) (1 1 1)   ⍝ signatures
's' 'QS s' Table (⍪ss) (⍪QS¨ss)
Assert (1 ¯1 0) ≡ QS 1 1 1

┌───────┬─────────┐
│s │QS s │
├───────┼─────────┤
│┌─────┐│┌───────┐│
││3 │││1 1 1 ││
│├─────┤│├───────┤│
││0 2 │││¯1 ¯1 ││
│├─────┤│├───────┤│
││3 0 1│││1 1 1 0││
│├─────┤│├───────┤│
││1 1 1│││1 ¯1 0 ││
│└─────┘│└───────┘│
└───────┴─────────┘

In [58]:
]dinput
_GA_ ← {
    _ ← Assert (a b ← ⍺)      VS (u v w ← ⍵)         ⍝ vector space: a b scalars, u v w vectors
    ∆ ← ⍺⍺                                           ⍝ geometric product
    s ← ⍵⍵                                           ⍝ signature
    b ← V¨BV+/s                                      ⍝ base unitary vectors, as multivectors
                                                     ⍝ properties:
    _ ← Assert (a ∆ v)        MV a × v               ⍝   left-product by scalar is scalar product
    _ ← Assert (v ∆ a)        MV v × a               ⍝   right-product by scalar is scalar product
    _ ← Assert (u ∆ v ∆ w)    MV w ∆⍨ u ∆ v          ⍝   associative product
    _ ← Assert (u ∆ v +_V w)  MV (u ∆ v) +_V u ∆ w   ⍝   distributive by left
    _ ← Assert (w ∆⍨ u +_V v) MV (u ∆ w) +_V v ∆ w   ⍝   distributive by right
    _ ← Assert (QS s)         MV  ∆⍨¨ b              ⍝   squared vectors given by signature
    _ ← Assert ∆/1,b                                 ⍝   non-zero product of base (orthonormal) vectors
    1
}

In [59]:
a b (Pq _GA_ 0 2) u v w

VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧


Quaternions are a signed geometric algebra.

## Signed geometric product

Instead of finding the multiplication table using multivectors with a potentially large number of additional dimensions, we will directly calculate it using the signature. The `BS` function returns the bub given the signature. We also define a new operator `_SQ` to find a multiplication factor for each bub product. We need to check if two bub's being multiplied contain the same vector (`⍺∧⍵`), and multiply by the corresponding squared value (given by `QS`). The `TS` function is analogous to `TD`, but it takes as argument a signature.

In [60]:
 BS ← BB(2*+/)                      ⍝ (B)inary unitary blades from (S)ignature
_SQ ← {(⍺SB⍵)×(⌽⍺⍺)×.*⍺∧⍵}          ⍝ (S)ign that multiplies product. ⍺⍺: s(Q)uared vectors, ⍺ ⍵ bub
 TS ← {(∘.PB,⍥⊂∘.((QS⍵)_SQ))⍨BS⍵}   ⍝ multiplication (T)able from (S)ignature
⍝ eg
'2' '0 2' '1 0 1' Table TS¨2 (0 2) (1 0 1)   ⍝ multiplication tables for some 2D algebras
'3' '1 1 1'       Table TS¨3 (1 1 1)         ⍝ multiplication tables for some 3D algebras

┌───────────────────┬────────────────────┬───────────────────┐
│2 │0 2 │1 0 1 │
├───────────────────┼────────────────────┼───────────────────┤
│┌───────┬─────────┐│┌───────┬──────────┐│┌───────┬─────────┐│
││0 1 2 3│1 1 1 1│││0 1 2 3│1 1 1 1│││0 1 2 3│1 1 1 1││
││1 0 3 2│1 1 ¯1 ¯1│││1 0 3 2│1 ¯1 ¯1 1│││1 0 3 2│1 1 ¯1 ¯1││
││2 3 0 1│1 1 1 1│││2 3 0 1│1 1 ¯1 ¯1│││2 3 0 1│1 1 0 0││
││3 2 1 0│1 1 ¯1 ¯1│││3 2 1 0│1 ¯1 1 ¯1│││3 2 1 0│1 1 0 0││
│└───────┴─────────┘│└───────┴──────────┘│└───────┴─────────┘│
└───────────────────┴────────────────────┴───────────────────┘

┌───────────────────────────────────────┬─────────────────────────────────────┐
│3 │1 1 1 │
├───────────────────────────────────────┼─────────────────────────────────────┤
│┌───────────────┬─────────────────────┐│┌───────────────┬───────────────────┐│
││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1│││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1││
││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1│││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││2 3 0 1 6 7 4 5│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││2 3 0 1 6 7 4 5│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││3 2 1 0 7 6 5 4│1 1 ¯1 ¯1 1 1 ¯1 ¯1│││3 2 1 0 7 6 5 4│1 1 1 1 1 1 1 1││
││4 5 6 7 0 1 2 3│1 1 1 1 1 1 1 1│││4 5 6 7 0 1 2 3│1 1 1 1 0 0 0 0││
││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 ¯1 ¯1 1 1│││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 0 0 0 0││
││6 7 4 5 2 3 0 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││6 7 4 5 2 3 0 1│1 1 ¯1 ¯1 0 0 0 0││
││7 6 5 4 3 2 1 0│1 1 ¯1 ¯1 1 1 ¯1 ¯1│││7 6 5 4 3 2 1 0│1 1 1 1 0 0 0 0││
│└───────────────┴─────────────────────┘│└───────────────┴───────────────────┘│
└───────────────────────────────────────┴─────────────────────────────────────┘

We can directly use the tables generated by `TS` with the `_∆_` operator previously defined:

In [61]:
_∆Q ← _∆_(TS 0 2)   ⍝ quaternion product operator
 ∆Q ← ×_∆Q          ⍝ quaternion product
i j ← V¨ 1 (0 1) ⋄ k ← i ∆Q j                 ⍝ quaternion units
'i' 'j' 'k' '∘.∆Q' Table i j k (∘.∆Q⍨i j k)   ⍝ quaternions multiplication table

┌───┬─────┬────────┬───────────────────────────┐
│i │j │k │∘.∆Q │
├───┼─────┼────────┼───────────────────────────┤
│0 1│0 0 1│0 0 0 ¯1│┌───────┬────────┬────────┐│
│ │ │ ││¯1 0 │0 0 0 ¯1│0 0 ¯1 0││
│ │ │ │├───────┼────────┼────────┤│
│ │ │ ││0 0 0 1│¯1 0 0 0│0 1 0 0 ││
│ │ │ │├───────┼────────┼────────┤│
│ │ │ ││0 0 1 0│0 ¯1 0 0│¯1 0 0 0││
│ │ │ │└───────┴────────┴────────┘│
└───┴─────┴────────┴───────────────────────────┘

And, now, it is easy to define different geometric algebras:

In [62]:
⍝ geometric algebras
Assert a b (∆Q _GA_ 0 2) i j k
Assert a b (×_∆_(TS 1 1 1)_GA_ 1 1 1) V¨ u v w
Assert a b (×_∆_(TS 3 0 1)_GA_ 3 0 1) V¨ u (v,1 RC 10) w

VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧
VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧
LENGTH ERROR: Mismatched left and right argument shapes
VS[9] _←Assert(a×v)E v×a           ⍝   commutative product with scalars
                      ∧


## Additional functions

### Exterior and inner products

There are other products that we can define, derived from the geometric product. The *exterior product* (which comes from [Grassman algebra](https://en.wikipedia.org/wiki/Exterior_algebra)) allows the formation of higher-dimensional entities when applied to orthonormal vectors, like the geometric product, but is always zero for parallel vectors. On the contrary, the *inner product* is equal to the geometric product for parallel vectors (and, therefore, equal to the dot product for proper vectors), but is zero for orthonormal vectors. The inner and exterior product are very useful to perform geometric operations.

We are going to define two operators. The monadic operator `_∆I` takes as operand a signature and applies the corresponding interior product (it directly uses `×` and `+/` instead of using the geometric product for simplicity). The dyadic operator `_∆X_` also takes a signature (as right argument), but also a product. It will apply the corresponding exterior product to the given arguments. It does this calculating a multiplication table for a signature in which all vectors are null vectors (therefore, their product will always be zero).

In [63]:
_∆X ← {⍺(×_∆_(TS 0 0,+/⍺(⍺⍺⍨⍨)⍵))⍵}    ⍝ e(X)terior product
_∆I ← {+/(0 0⍉⊃⌽TS⍺(⍺⍺⍨⍨)⍵)×_X⍺×_X⍵}   ⍝ (I)nner product
∆X ← D _∆X                             ⍝ exterior 3D product
∆I ← D _∆I                             ⍝ inner 3D product
⍝ eg
'∘.∆I' '∘.∆X' Table (∘.∆I⍨ V¨ x y z) (∘.∆X⍨ V¨ x y z)
'∘.(1 1 1 _∆I)' '∘.(1 1 1 _∆X)' Table (∘.(1 1 1 _∆I)⍨ V¨ x y z) (∘.(1 1 1 _∆X)⍨ V¨ x y z)

┌─────┬───────────────────────────────────────────┐
│∘.∆I │∘.∆X │
├─────┼───────────────────────────────────────────┤
│1 0 0│┌───────────┬─────────────┬───────────────┐│
│0 1 0││0 0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 ││
│0 0 1│├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 1 │0 0 0 0 │0 0 0 0 0 0 ¯1 ││
│ │├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 0││
│ │└───────────┴─────────────┴───────────────┘│
└─────┴───────────────────────────────────────────┘

┌─────────────┬───────────────────────────────────────────┐
│∘.(1 1 1 _∆I)│∘.(1 1 1 _∆X) │
├─────────────┼───────────────────────────────────────────┤
│1 0 0 │┌───────────┬─────────────┬───────────────┐│
│0 ¯1 0 ││0 0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 ││
│0 0 0 │├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 1 │0 0 0 0 │0 0 0 0 0 0 ¯1 ││
│ │├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 0││
│ │└───────────┴─────────────┴───────────────┘│
└─────────────┴───────────────────────────────────────────┘

Alternatively, we could have defined the inner and exterior products in basis of the geometric product:

In [64]:
Assert ∘.((∆I MV 2÷⍨∆+_V ∆⍨)⍥V)⍨ u v w
Assert ∘.((∆X MV 2÷⍨∆-_V ∆⍨)⍥V)⍨ u v w

VALUE ERROR: Undefined name: MV
      Assert∘.((∆I MV 2÷⍨∆+_V ∆⍨)⍥V)⍨u v w
                   ∧
VALUE ERROR: Undefined name: MV
      Assert∘.((∆X MV 2÷⍨∆-_V ∆⍨)⍥V)⍨u v w
                   ∧


The exterior and inner products, when added together, define the geometric product:

In [65]:
Assert ∘.((∆ MV ∆I +_V ∆X)⍥V)⍨ u v w

VALUE ERROR: Undefined name: MV
      Assert∘.((∆ MV ∆I+_V ∆X)⍥V)⍨u v w
                  ∧


So that we can use precomputed tables

In [66]:
⍝ inner product
 TI ← 0 0⍉∘⊃⌽∘TS           ⍝ (T)able for (I)nner product
_∆I ← {+/⍺(⍺⍺×_X×_X)⍵}     ⍝ (I)nner product, ⍺⍺ is table
 ∆I ← TI⍤D _∆I             ⍝ (I)nner product
⍝ exterior product
 TX  ← TS 0 0,+/           ⍝ (T)able for e(X)terior product
_∆X_ ← {⍺(⍺⍺_∆_(TX⍵⍵))⍵}   ⍝ e(X)terior product, ⍺⍺ is product function, ⍵⍵ is signature
_∆X  ← _∆_(TX⍤D)           ⍝ e(X)terior product, ⍺⍺ is product function
 ∆X  ← ×_∆X                ⍝ e(X)terior product
⍝ eg
'∘.∆I' '∘.∆X' Table (∘.∆I⍨ V¨ x y z) (∘.∆X⍨ V¨ x y z)
'∘.(1 1 1 _∆I)' '∘.(×_∆X_ 1 1 1)' Table (∘.(1 1 1 _∆I)⍨ V¨ x y z) (∘.(×_∆X_ 1 1 1)⍨ V¨ x y z)
Assert ∘.((∆I MV 2÷⍨∆+_V ∆⍨)⍥V)⍨ u v w
Assert ∘.((∆X MV 2÷⍨∆-_V ∆⍨)⍥V)⍨ u v w
Assert ∘.((∆  MV  ∆I +_V ∆X)⍥V)⍨ u v w

┌─────┬───────────────────────────────────────────┐
│∘.∆I │∘.∆X │
├─────┼───────────────────────────────────────────┤
│1 0 0│┌───────────┬─────────────┬───────────────┐│
│0 1 0││0 0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 ││
│0 0 1│├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 1 │0 0 0 0 │0 0 0 0 0 0 ¯1 ││
│ │├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 0││
│ │└───────────┴─────────────┴───────────────┘│
└─────┴───────────────────────────────────────────┘

┌─────────────┬───────────────────────────────────────────┐
│∘.(1 1 1 _∆I)│∘.(×_∆X_ 1 1 1) │
├─────────────┼───────────────────────────────────────────┤
│1 0 0 │┌───────────┬─────────────┬───────────────┐│
│0 1 0 ││0 0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 ││
│0 0 0 │├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 1 │0 0 0 0 │0 0 0 0 0 0 ¯1 ││
│ │├───────────┼─────────────┼───────────────┤│
│ ││0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 0││
│ │└───────────┴─────────────┴───────────────┘│
└─────────────┴───────────────────────────────────────────┘

VALUE ERROR: Undefined name: MV
      Assert∘.((∆I MV 2÷⍨∆+_V ∆⍨)⍥V)⍨u v w
                   ∧
VALUE ERROR: Undefined name: MV
      Assert∘.((∆X MV 2÷⍨∆-_V ∆⍨)⍥V)⍨u v w
                   ∧
VALUE ERROR: Undefined name: MV
      Assert∘.((∆ MV ∆I+_V ∆X)⍥V)⍨u v w
                  ∧


In [67]:
⍝ inner product
 TI ← 0 0⍉∘⊃⌽∘TS                ⍝ (T)able for (I)nner product
_∆I_ ← {0 V _X⍺(⍵⍵×_X⍺⍺¨_X)⍵}   ⍝ (I)nner product, ⍺⍺ is multiplication, ⍵⍵ is table
_∆I  ← _∆I_(TI⍤D)               ⍝ (I)nner product, ⍺⍺ is multiplication, ⍵⍵ is table
 ∆I  ← ×_∆I                     ⍝ (I)nner product
⍝ exterior product
 TX  ← TS 0 0,+/                ⍝ (T)able for e(X)terior product
_∆X_ ← {⍺(⍺⍺_∆_(TX⍵⍵))⍵}        ⍝ e(X)terior product, ⍺⍺ is product function, ⍵⍵ is signature
_∆X  ← _∆_(TX⍤D)                ⍝ e(X)terior product, ⍺⍺ is product function
 ∆X  ← ×_∆X                     ⍝ e(X)terior product
⍝ eg
'∘.∆I' '∘.∆X' Table (∘.∆I⍨ V¨ x y z) (∘.∆X⍨ V¨ x y z)
⍝'∘.(1 1 1 _∆I)' '∘.(×_∆X_ 1 1 1)' Table (∘.(1 1 1 _∆I)⍨ V¨ x y z) (∘.(×_∆X_ 1 1 1)⍨ V¨ x y z)
Assert ∘.((∆I MV 2÷⍨∆+_V ∆⍨)⍥V)⍨ u v w
Assert ∘.((∆X MV 2÷⍨∆-_V ∆⍨)⍥V)⍨ u v w
Assert ∘.((∆  MV  ∆I +_V ∆X)⍥V)⍨ u v w

┌───────┬───────────────────────────────────────────┐
│∘.∆I │∘.∆X │
├───────┼───────────────────────────────────────────┤
│┌─┬─┬─┐│┌───────────┬─────────────┬───────────────┐│
││1│0│0│││0 0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 ││
│├─┼─┼─┤│├───────────┼─────────────┼───────────────┤│
││0│1│0│││0 0 0 1 │0 0 0 0 │0 0 0 0 0 0 ¯1 ││
│├─┼─┼─┤│├───────────┼─────────────┼───────────────┤│
││0│0│1│││0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 0││
│└─┴─┴─┘│└───────────┴─────────────┴───────────────┘│
└───────┴───────────────────────────────────────────┘

VALUE ERROR: Undefined name: MV
      Assert∘.((∆I MV 2÷⍨∆+_V ∆⍨)⍥V)⍨u v w
                   ∧
VALUE ERROR: Undefined name: MV
      Assert∘.((∆X MV 2÷⍨∆-_V ∆⍨)⍥V)⍨u v w
                   ∧
VALUE ERROR: Undefined name: MV
      Assert∘.((∆ MV ∆I+_V ∆X)⍥V)⍨u v w
                  ∧


### Reverses and complements

The [reverse](http://rigidgeometricalgebra.org/wiki/index.php?title=Reverses) is a linear operation, so it can be applied component by component. The reverse of a blade of grade n that constitutes the product of n 1-vectors is the result of multiplying those same n 1-vectors in the reverse order (from right to left instead of from left to right). The result is that each blade must be multiplied by `1` or by `¯1`. This can be calculated simply as the result of squaring each blade (`SB⍨BB`):

In [68]:
R ← ×∘(SB⍨¨BB∘≢)⍨   ⍝ (R)everse
⍝ eg
i←0.1,⍎¨1↓(↓↑BB 16)(⊢⍤/¨)⊂'4321'
'i' 'R i' Table (⍪¨B)¨ i (R i)
Assert MV∘R∘R⍨¨ i u v w   ⍝ reverse of reverse is the same

┌───────────────────┬─────────────────────┐
│i │R i │
├───────────────────┼─────────────────────┤
│┌───┬─┬──┬───┬────┐│┌───┬─┬───┬────┬────┐│
││0.1│1│21│321│4321│││0.1│1│¯21│¯321│4321││
││ │2│31│421│ │││ │2│¯31│¯421│ ││
││ │3│32│431│ │││ │3│¯32│¯431│ ││
││ │4│41│432│ │││ │4│¯41│¯432│ ││
││ │ │42│ │ │││ │ │¯42│ │ ││
││ │ │43│ │ │││ │ │¯43│ │ ││
│└───┴─┴──┴───┴────┘│└───┴─┴───┴────┴────┘│
└───────────────────┴─────────────────────┘

VALUE ERROR: Undefined name: MV
      Assert MV∘R∘R⍨¨i u v w   ⍝ reverse of reverse is the same
             ∧


Like reverses, [complements](http://rigidgeometricalgebra.org/wiki/index.php?title=Complements) are linear operations. The left complement of a unitary blade is the unitary blade that multiplied by the left will result in the *pseudoscalar*. The pseudoscalar for n dimensions is the last blade, corresponding to the product of n 1-vectors. Its right complement is the unitary blade that multiplied by the right will result in the pseudoscalar. And the double complement can be calculated either as the result of applying twice the left complement or as the result of applying twice the right complement.

Before defining the functions to find the complement, we define an operator `_C` which calculates the products using the given function. The left and righ complements are then found using as operand either the geometric product, or its reverse:

In [69]:
_C ← {⍺←D⍵ ⋄ (⌽(2*⍺)X⍵)×_V+/⍤⍺⍺¨∘⌽⍨BV≢(2*⍺)X⍵}   ⍝ (C)omplement
LC ← ∆ _C                                        ⍝ (L)eft complement
RC ← ∆⍨_C                                        ⍝ (R)ight complement
DC ← LC⍣2                                        ⍝ (D)ouble complement
⍝ eg
'i' 'LC i' 'RC i' 'DC i' Table (⍪¨B)¨ i (LC i) (R i) (DC i)
p  ← (1↑⍨∘-2*∘⌈2⍟≢)¨bu          ⍝ pseudoscalars
Assert p MV∘(∆⍨∘LC⍨)¨bu         ⍝ Left-product by LC is pseudoscalar
Assert p MV∘(∆∘RC⍨)¨bu          ⍝ right-product by LC is pseudoscalar
Assert (zyx MV ∆⍨∘(3∘LC)⍨)¨bu   ⍝ left-product by 8∘LC is the 3D pseudoscalar
Assert (zyx MV ∆∘(3∘LC)⍨)¨bu    ⍝ right-product by 8∘LC is the 3D pseudoscalar
Assert MV∘RC∘LC⍨¨ u v w         ⍝ right complement of left complement is the same
Assert (DC MV RC⍣2)¨ u v w      ⍝ double left complement and double right complement are the same

┌───────────────────┬──────────────────────┬─────────────────────┬─────────────────────┐
│i │LC i │RC i │DC i │
├───────────────────┼──────────────────────┼─────────────────────┼─────────────────────┤
│┌───┬─┬──┬───┬────┐│┌────┬────┬───┬──┬───┐│┌───┬─┬───┬────┬────┐│┌───┬──┬──┬────┬────┐│
││0.1│1│21│321│4321│││4321│¯432│ 43│¯4│0.1│││0.1│1│¯21│¯321│4321│││0.1│¯1│21│¯321│4321││
││ │2│31│421│ │││ │ 431│¯42│ 3│ │││ │2│¯31│¯421│ │││ │¯2│31│¯421│ ││
││ │3│32│431│ │││ │¯421│ 41│¯2│ │││ │3│¯32│¯431│ │││ │¯3│32│¯431│ ││
││ │4│41│432│ │││ │ 321│ 32│ 1│ │││ │4│¯41│¯432│ │││ │¯4│41│¯432│ ││
││ │ │42│ │ │││ │ │¯31│ │ │││ │ │¯42│ │ │││ │ │42│ │ ││
││ │ │43│ │ │││ │ │ 21│ │ │││ │ │¯43│ │ │││ │ │43│ │ ││
│└───┴─┴──┴───┴────┘│└────┴────┴───┴──┴───┘│└───┴─┴───┴────┴────┘│└───┴──┴──┴────┴────┘│
└───────────────────┴──────────────────────┴─────────────────────┴─────────────────────┘

VALUE ERROR: Undefined name: MV
      Assert p MV∘(∆⍨∘LC⍨)¨bu         ⍝ Left-product by LC is pseudoscalar
               ∧
VALUE ERROR: Undefined name: MV
      Assert p MV∘(∆∘RC⍨)¨bu          ⍝ right-product by LC is pseudoscalar
               ∧
VALUE ERROR: Undefined name: MV
      Assert(zyx MV ∆⍨∘(3∘LC)⍨)¨bu   ⍝ left-product by 8∘LC is the 3D pseudoscalar
                 ∧
VALUE ERROR: Undefined name: MV
      Assert(zyx MV ∆∘(3∘LC)⍨)¨bu    ⍝ right-product by 8∘LC is the 3D pseudoscalar
                 ∧
VALUE ERROR: Undefined name: MV
      Assert MV∘RC∘LC⍨¨u v w         ⍝ right complement of left complement is the same
             ∧
VALUE ERROR: Undefined name: MV
      Assert(DC MV RC⍣2)¨u v w      ⍝ double left complement and double right complement are the same
                ∧


In [70]:
_TC ← {+/⍤⍺⍺¨∘⌽⍨⍤BV≢⍵}          ⍝ (T)able of (C)omplements. ⍺⍺: product
_C  ← {⍺←D⍵ ⋄ (⍺⍺×_V⌽)⍵X⍨2*⍺}   ⍝ (C)omplement. ⍺⍺: table
LC ← ∆ _TC _C                   ⍝ (L)eft complement
RC ← ∆⍨_TC _C                   ⍝ (R)ight complement
DC ← LC⍣2                       ⍝ (D)ouble complement
⍝ eg
'i' 'LC i' 'RC i' 'DC i' Table (⍪¨B)¨ i (LC i) (R i) (DC i)
p ← (1↑⍨∘-2*D)¨bu               ⍝ pseudoscalars
Assert p MV∘(∆⍨∘LC⍨)¨bu         ⍝ Left-product by LC is pseudoscalar
Assert p MV∘(∆∘RC⍨)¨bu          ⍝ right-product by LC is pseudoscalar
Assert (zyx MV ∆⍨∘(3∘LC)⍨)¨bu   ⍝ left-product by 8∘LC is the 3D pseudoscalar
Assert (zyx MV ∆∘(3∘LC)⍨)¨bu    ⍝ right-product by 8∘LC is the 3D pseudoscalar
Assert MV∘RC∘LC⍨¨ u v w         ⍝ right complement of left complement is the same
Assert (DC MV RC⍣2)¨ u v w      ⍝ double left complement and double right complement are the same

┌───────────────────┬──────────────────────┬─────────────────────┬─────────────────────┐
│i │LC i │RC i │DC i │
├───────────────────┼──────────────────────┼─────────────────────┼─────────────────────┤
│┌───┬─┬──┬───┬────┐│┌────┬────┬───┬──┬───┐│┌───┬─┬───┬────┬────┐│┌───┬──┬──┬────┬────┐│
││0.1│1│21│321│4321│││4321│¯432│ 43│¯4│0.1│││0.1│1│¯21│¯321│4321│││0.1│¯1│21│¯321│4321││
││ │2│31│421│ │││ │ 431│¯42│ 3│ │││ │2│¯31│¯421│ │││ │¯2│31│¯421│ ││
││ │3│32│431│ │││ │¯421│ 41│¯2│ │││ │3│¯32│¯431│ │││ │¯3│32│¯431│ ││
││ │4│41│432│ │││ │ 321│ 32│ 1│ │││ │4│¯41│¯432│ │││ │¯4│41│¯432│ ││
││ │ │42│ │ │││ │ │¯31│ │ │││ │ │¯42│ │ │││ │ │42│ │ ││
││ │ │43│ │ │││ │ │ 21│ │ │││ │ │¯43│ │ │││ │ │43│ │ ││
│└───┴─┴──┴───┴────┘│└────┴────┴───┴──┴───┘│└───┴─┴───┴────┴────┘│└───┴──┴──┴────┴────┘│
└───────────────────┴──────────────────────┴─────────────────────┴─────────────────────┘

VALUE ERROR: Undefined name: MV
      Assert p MV∘(∆⍨∘LC⍨)¨bu         ⍝ Left-product by LC is pseudoscalar
               ∧
VALUE ERROR: Undefined name: MV
      Assert p MV∘(∆∘RC⍨)¨bu          ⍝ right-product by LC is pseudoscalar
               ∧
VALUE ERROR: Undefined name: MV
      Assert(zyx MV ∆⍨∘(3∘LC)⍨)¨bu   ⍝ left-product by 8∘LC is the 3D pseudoscalar
                 ∧
VALUE ERROR: Undefined name: MV
      Assert(zyx MV ∆∘(3∘LC)⍨)¨bu    ⍝ right-product by 8∘LC is the 3D pseudoscalar
                 ∧
VALUE ERROR: Undefined name: MV
      Assert MV∘RC∘LC⍨¨u v w         ⍝ right complement of left complement is the same
             ∧
VALUE ERROR: Undefined name: MV
      Assert(DC MV RC⍣2)¨u v w      ⍝ double left complement and double right complement are the same
                ∧


### Antiproducts

[De Morgan laws](https://rigidgeometricalgebra.org/wiki/index.php?title=Geometric_products#De_Morgan_Laws)

In [71]:
_⍙_ ← {⍵⍵ LC ⍺ ⍺⍺⍥((⍺(⍵⍵⍨⍨)⍵)∘RC) ⍵}   ⍝ ⍺⍺ is product, ⍵⍵ is dimensions
⍙ ← ∆ _⍙_ 3
Assert (3 LC u ⍙ v) MV u ∆⍥(3∘LC) v
Assert (3 RC u ∆ v) MV u ⍙⍥(3∘RC) v
Assert (3 RC u ⍙ v) MV u ∆⍥(3∘RC) v
∆I ← 3 _∆I ⋄ ⍙I ← ∆I _⍙_ 3
Assert (3 LC u ⍙I v) MV u ∆I⍥(3∘LC) v
Assert (3 RC u ∆I v) MV u ⍙I⍥(3∘RC) v
Assert (3 RC u ⍙I v) MV u ∆I⍥(3∘RC) v
∆X ← ×_∆X_ 3 ⋄ ⍙X ← ∆X _⍙_ 3
Assert (3 LC u ⍙X v) MV u ∆X⍥(3∘LC) v
Assert (3 RC u ∆X v) MV u ⍙X⍥(3∘RC) v
Assert (3 RC u ⍙X v) MV u ∆X⍥(3∘RC) v

VALUE ERROR: Undefined name: MV
      Assert(3 LC u ⍙ v)MV u ∆⍥(3∘LC)v
                        ∧
VALUE ERROR: Undefined name: MV
      Assert(3 RC u ∆ v)MV u ⍙⍥(3∘RC)v
                        ∧
VALUE ERROR: Undefined name: MV
      Assert(3 RC u ⍙ v)MV u ∆⍥(3∘RC)v
                        ∧
VALUE ERROR: Undefined name: MV
      Assert(3 LC u ⍙I v)MV u ∆I⍥(3∘LC)v
                         ∧
VALUE ERROR: Undefined name: MV
      Assert(3 RC u ∆I v)MV u ⍙I⍥(3∘RC)v
                         ∧
VALUE ERROR: Undefined name: MV
      Assert(3 RC u ⍙I v)MV u ∆I⍥(3∘RC)v
                         ∧
VALUE ERROR: Undefined name: MV
      Assert(3 LC u ⍙X v)MV u ∆X⍥(3∘LC)v
                         ∧
VALUE ERROR: Undefined name: MV
      Assert(3 RC u ∆X v)MV u ⍙X⍥(3∘RC)v
                         ∧
VALUE ERROR: Undefined name: MV
      Assert(3 RC u ⍙X v)MV u ∆X⍥(3∘RC)v
                         ∧


### Inverse

Find the multiversor that, when multiplied by the right, returns the scalar unit. At difference of reverses and complements, the inverse is not a linear operation.

In [72]:
_I ← {i←Z∘C⊂⍤¯1⍉⍤¯1⍉((1↑⍨≢)⌹⍤2(↑[¯0.5]⍣2⊂⍺⍺¨BV∘≢))(2*D⍵)X⍵ ⋄ 0=⎕NC'⍺':i ⋄ ⍺ ⍺⍺ i}
∆∆ ← ∆ _I

' ' '∆∆' '∆∆⍨' Table (⍪¨B i)(⍪¨B ∆∆ i)(∆∆⍨i)
Assert 1 MV¨ ∆∆⍨¨   u v w
Assert 1 MV¨ ∆∆⍨∘V¨ u v w

┌───────────────────┬──────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ │∆∆ │∆∆⍨ │
├───────────────────┼──────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│┌───┬─┬──┬───┬────┐│┌────────┬─────────┬────────────┬──────────┬─────────┐│1 2.776E¯17 ¯4.163E¯17 3.469E¯18 1.388E¯17 ¯3.469E¯18 3.469E¯18 0 2.776E¯17 8.674E¯19 0 1.084E¯19 ¯1.735E¯18 1.084E¯19 2.168E¯19 ¯1.355E¯20│
││0.1│1│21│321│4321│││1.821E¯8│¯1.830E¯7│¯0.000001179│0.00001781│0.0002397││ │
││ │2│31│421│ │││ │¯1.238E¯7│¯0.000001708│0.00002336│ ││ │
││ │3│32│431│ │││ │¯1.770E¯7│¯0.000001773│0.00002391│ ││ │
││ │4│41│432│ │││ │¯2.342E¯7│¯0.000002285│0.00002397│ ││ │
││ │ │42│ │ │││ │ │¯0.00000233 │ │ ││ │
││ │ │43│ │ │││ │ │¯0.000002387│ │ ││ │
│└───┴─┴──┴───┴────┘│└────────┴─────────┴────────────┴──────────┴─────────┘│ │
└───────────────────┴──────────────────────────────────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

VALUE ERROR: Undefined name: MV
      Assert 1 MV¨∆∆⍨¨u v w
               ∧
VALUE ERROR: Undefined name: MV
      Assert 1 MV¨∆∆⍨∘V¨u v w
               ∧


When we try to inverse a 0=vector, we will get a domain error.

In [73]:
∆∆ n   ⍝ ERROR: 0=vectors produce domain error

DOMAIN ERROR
_I[0] _I←{i←Z∘C⊂⍤¯1⍉⍤¯1⍉((1↑⍨≢)⌹⍤2(↑[¯0.5]⍣2⊂⍺⍺¨BV∘≢))(2*D ⍵)X ⍵ ⋄ 0=⎕NC'⍺':i ⋄ ⍺ ⍺⍺ i}
                        ∧


## Geometric algebra namespace

Since it is clear that using a fixed multiplication table has several advantages, and in order to make it easier to define a geometric product, we are going to use a namespace to store the properties of the algebra and all related functions.

The function `G` returns a namespace that contains a geometric product operator and a geometric product function, as well as the signature and multiplication table of the algebra.

In [74]:
]dinput
GA ← {
    g ← ⎕NS⍬ ⋄ g.s g.t g.b ← ⍵ (TS⍵) (BV 2*+/⍵)
    g._Z ← _Z ⋄ g.V ← V ⋄ g.Z ← Z ⋄ g.S ← S ⋄ g.D ← D ⋄ g.BV ← BV ⋄ g.C ← C
    g.X ← X{⍺←≢b ⋄ ⍺(⍺⍺)⍵}
    
    g._∆ ← _∆_ g.t ⋄ g.∆ ← ×g._∆ ⋄ g.∆∆ ← g.∆ _I
    g.∆V ← g.∆⍥V ⋄ g.∆v ← g.∆∘V ⋄ g.∆∆V ← g.∆∆⍥V ⋄ g.∆∆v ← g.∆∆∘V
    g.LC ← g.∆ _C ⋄ g.RC ← g.∆⍨_C ⋄ g.DC ← g.LC⍣2
    g._∆X ← _∆X_ g.s ⋄ g.∆X ← ×g._∆X ⋄ g.∆XV ← g.∆X⍥V ⋄ g.∆Xv ← g.∆X∘V
    g.∆I ← g.s _∆I ⋄ g.∆IV ← g.∆I⍥V ⋄ g.∆Iv ← g.∆I∘V
    g.⍙ ← g.∆ _⍙_(+/g.s) ⋄ g.⍙I ← g.∆I _⍙_(+/g.s) ⋄ g.⍙X ← g.∆X _⍙_(+/g.s)
    g
}

In [75]:
⍝ eg
g3 ← GA 3
Assert a b (g3.∆ _GA_ g3.s) V¨ u v w
'sign' 'table' 'base' Table g3.s g3.t (⍪g3.b)

VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧


┌────┬───────────────────────────────────────┬─────────────────┐
│sign│table │base │
├────┼───────────────────────────────────────┼─────────────────┤
│3 │┌───────────────┬─────────────────────┐│┌───────────────┐│
│ ││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1│││1 ││
│ ││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1││├───────────────┤│
│ ││2 3 0 1 6 7 4 5│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││0 1 ││
│ ││3 2 1 0 7 6 5 4│1 1 ¯1 ¯1 1 1 ¯1 ¯1││├───────────────┤│
│ ││4 5 6 7 0 1 2 3│1 1 1 1 1 1 1 1│││0 0 1 ││
│ ││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 ¯1 ¯1 1 1││├───────────────┤│
│ ││6 7 4 5 2 3 0 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││0 0 0 1 ││
│ ││7 6 5 4 3 2 1 0│1 1 ¯1 ¯1 1 1 ¯1 ¯1││├───────────────┤│
│ │└───────────────┴─────────────────────┘││0 0 0 0 1 ││
│ │ │├───────────────┤│
│ │ ││0 0 0 0 0 1 ││
│ │ │├───────────────┤│
│ │ ││0 0 0 0 0 0 1 ││
│ │ │├───────────────┤│
│ │ ││0 0 0 0 0 0 0 1││
│ │ │└───────────────┘│
└────┴───────────────────────────────────────┴─────────────────┘

# Geometric algebras

## 0D / 1D / 2D / 3D vectors

We can easily define algebras for vector spaces of an arbitrary number of dimensions.

The 0-dimensional vector space will be the scalar field (usually the real numbers; in APL, just numbers). The inverse and multiplication function must work as `÷` and `×`, and using enclosed arrays, scalar extension should be applied:

In [76]:
g0 ← GA 0
'a' 'b' 'a g0.∆ b' 'a g0.∆∆ b' Table a b (a g0.∆ b) (a g0.∆∆ b)
Assert a b (g0.∆ _GA_ 0) ⊂¨v
Assert a (⊂⍤× MV g0.∆) b
Assert a (⊂⍤÷ MV g0.∆∆) b

┌────────┬───────┬────────┬─────────┐
│a │b │a g0.∆ b│a g0.∆∆ b│
├────────┼───────┼────────┼─────────┤
│┌──────┐│┌─────┐│┌──────┐│┌───────┐│
││3.794 │││3.535│││13.41 │││1.073 ││
││0.2032│││4.926│││ 1.001│││0.04125││
││3.971 │││4.397│││17.46 │││0.9031 ││
││2.502 │││4.684│││11.72 │││0.5343 ││
││1.006 │││4.943│││ 4.974│││0.2036 ││
│└──────┘│└─────┘│└──────┘│└───────┘│
└────────┴───────┴────────┴─────────┘

VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧
VALUE ERROR: Undefined name: MV
      Assert a(⊂⍤×MV g0.∆)b
                  ∧
VALUE ERROR: Undefined name: MV
      Assert a(⊂⍤÷MV g0.∆∆)b
                  ∧


When we add a proper vector to have the 1-dimensional space, we will in fact have the [hyperbolic numbers](https://en.wikipedia.org/wiki/Split-complex_number), with these properties:

In [77]:
g1 ← GA 1
'a' 'b' 'g1.∆⍨ a b' Table a b (g1.∆⍨ a b)
Assert a b (g1.∆ _GA_ 1) 2↑¨ u v w

┌────────┬───────┬──────────────────┐
│a │b │g1.∆⍨ a b │
├────────┼───────┼──────────────────┤
│┌──────┐│┌─────┐│┌───────┬────────┐│
││3.794 │││3.535│││┌─────┐│┌──────┐││
││0.2032│││4.926││││26.89│││26.82 │││
││3.971 │││4.397││││24.31│││ 2.002│││
││2.502 │││4.684││││35.1 │││34.91 │││
││1.006 │││4.943││││28.2 │││23.44 │││
│└──────┘│└─────┘│││25.44│││ 9.948│││
│ │ ││└─────┘│└──────┘││
│ │ │└───────┴────────┘│
└────────┴───────┴──────────────────┘

VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧


With two dimensions, things start getting interesting. Now we have vectors on a plane, and we can rotate them!

In [78]:
g2 ← GA 2
Assert a b (g2.∆ _GA_ 2) u v w
r90 ← 0 1   g2.∆∆⍥V 1 ⋄ Assert (V 0 1)   MV r90 g2.∆   V 1   ⍝   0 1 is 90 deg rotation of 1 0
r45 ← 1 1 U⍤g2.∆∆⍥V 1 ⋄ Assert (V U 1 1) MV r45 g2.∆   V 1   ⍝ U 1 1 is 45 deg rotation of 1 0 
                        Assert (V 0 1)   MV r45 g2.∆⍣2 V 1   ⍝   0 1 is 2 45 deg rotations of 1 0

VALUE ERROR: Undefined name: MV
_GA_[6] _←Assert(a ∆ v)MV a×v               ⍝   left-product by scalar is scalar product
                       ∧
VALUE ERROR: Undefined name: MV
      r90←0 1 g2.∆∆⍥V 1 ⋄ Assert(V 0 1)MV r90 g2.∆ V 1   ⍝   0 1 is 90 deg rotation of 1 0
                                       ∧
VALUE ERROR: Undefined name: MV
      r45←1 1 U⍤g2.∆∆⍥V 1 ⋄ Assert(V U 1 1)MV r45 g2.∆ V 1   ⍝ U 1 1 is 45 deg rotation of 1 0
                                           ∧
VALUE ERROR: Undefined name: MV
      Assert(V 0 1)MV r45 g2.∆⍣2 V 1   ⍝   0 1 is 2 45 deg rotations of 1 0
                   ∧


We can also use bivectors as areas:

In [79]:
'a' 'b' 's←a g2.∆⍥V 0,b' 's g2.∆∆∘V 0,b' Table a b (a g2.∆V 0,b) ((0,b) g2.∆∆∘V⍨ a g2.∆V 0,b)

┌────────┬───────┬───────────────┬──────────────┐
│a │b │s←a g2.∆⍥V 0,b │s g2.∆∆∘V 0,b │
├────────┼───────┼───────────────┼──────────────┤
│┌──────┐│┌─────┐│┌─┬─┬─┬───────┐│┌─┬──────┬─┬─┐│
││3.794 │││3.535│││0│0│0│¯13.41 │││0│3.794 │0│0││
││0.2032│││4.926│││ │ │ │ ¯1.001│││ │0.2032│ │ ││
││3.971 │││4.397│││ │ │ │¯17.46 │││ │3.971 │ │ ││
││2.502 │││4.684│││ │ │ │¯11.72 │││ │2.502 │ │ ││
││1.006 │││4.943│││ │ │ │ ¯4.974│││ │1.006 │ │ ││
│└──────┘│└─────┘│└─┴─┴─┴───────┘│└─┴──────┴─┴─┘│
└────────┴───────┴───────────────┴──────────────┘

The 3D space is pretty much where we live, and has already been discussed in a few examples, so you should be familiar with this one by now. We have scalars, vectors, bivectors, and the pseudoscalar is a trivector:

In [80]:
B 0 1 2 21 3 31 32 321 ⍝ 1 x y yx z zx zy zyx

┌─┬─────┬────────┬───┐
│0│1 2 3│21 31 32│321│
└─┴─────┴────────┴───┘

We can use the quaternions (scalars and bivectors) to rotate vectors or other quaternions, and we can use bivectors to represent areas and the trivector for volumes:

## Complex numbers

Complex numbers are obtained by the addition of a real number (a scalar in APL) and an imaginary number. Although Dyalog APL supports complex numbers, we can also represent them defining a geometric algebra with the signature 0 1:

In [81]:
c ← GA 0 1
'C' 'GA 0 1' Table (∘.×⍨ 1 0j1) (∘.c.∆⍨ c.b)
Assert (∘.×⍨ 1 0j1) ≡ (⊂1 0j1)+.×∘c.X¨∘.c.∆⍨ c.b
'c' '∆∆ c' '∆∆⍨c' Table (c.X v) (∆∆ c.X v) (∆∆⍨c.X v)
Assert (9 11○1÷1j1) ≡ c.∆∆ 1 1
Assert (9 11○1 ÷ 1 0j1 +.× c.X v) ≡ c.∆∆ c.X v

┌────────┬──────────┐
│C │GA 0 1 │
├────────┼──────────┤
│1 0J1│┌───┬────┐│
│0J1 ¯1 ││1 │0 1 ││
│ │├───┼────┤│
│ ││0 1│¯1 0││
│ │└───┴────┘│
└────────┴──────────┘

┌──────────────┬──────────────────┬──────────────┐
│c │∆∆ c │∆∆⍨c │
├──────────────┼──────────────────┼──────────────┤
│┌─────┬──────┐│┌───────┬────────┐│┌─┬──────────┐│
││2.091│2.395 │││¯1.535 │ 1.758 │││1│ 0.000E0 ││
││2.992│1.062 │││ 0.3824│¯0.1357 │││1│¯5.551E¯17││
││2.048│0.2925│││ 0.4986│¯0.07123│││1│¯2.776E¯17││
││1.754│2.754 │││¯0.389 │ 0.6108 │││1│ 0.000E0 ││
││1.887│2.398 │││¯0.8605│ 1.094 │││1│ 0.000E0 ││
│└─────┴──────┘│└───────┴────────┘│└─┴──────────┘│
└──────────────┴──────────────────┴──────────────┘

## Quaternions

Dyalog APL does not support quaternions natively, but an implementation is included in the dfns workspace.

In [82]:
_∆Q3 ← {⊃,/0 2∘B⍺(⍺⍺_∆_(TD 3)⍥(0 3 5 6∘V))⍵}
_∆Q3 ← {Z 1 1 1 ¯1×_X⊃,/0 2 B 8 X ⍺(⍺⍺_∆_(TD 3)⍥(0 3 5 6 V 1 1 1 ¯1 ×_X ⊢))⍵}
∆Q3 ← ×_∆Q3
(∘.∆Q3⍨BV 4) ≡ (∘.∆Q⍨BV 4)

0

In [83]:
('ijk',⊂'∘.(×H)') Table (⊂∘.(×H)⍨ i j k),⍨i j k ← ↓ ⍎H'0i1 0j1 0k1'
Assert (∘.(×H)⍨ i j k) ≡ (⊂1 1 1 ¯1)×_X¨∘.∆Q⍨ i j (-k)
⍝_∆Q3 ← {,/0 2∘B⍺(⍺⍺_∆_(TD 3)⍥(0 3 5 6∘V))⍵}
⍝∆Q3 ← ×_∆q3
qg ← ⊂⍤1⍉ qh ← ?1e4 4⍴0
]RunTime 'qg ∆Q  qg' 'qg ∆Q3 qg' 'qh ×H  qh' -compare
qg ← ⊂⍤1⍉ qh ← ?1e1 4⍴0
]RunTime 'qg ∘.×_∆Q  qg' 'qg ∘.×_∆Q3 qg' 'qh ∘.(×H)  qh' -compare
]SpaceNeeded 'qg ∘.×_∆Q  qg' 'qg ∘.×_∆Q3 qg' 'qh ∘.(×H)  qh'

┌───────┬───────┬───────┬────────────────────────────┐
│i │j │k │∘.(×H) │
├───────┼───────┼───────┼────────────────────────────┤
│0 1 0 0│0 0 1 0│0 0 0 1│┌────────┬────────┬────────┐│
│ │ │ ││¯1 0 0 0│0 0 0 1 │0 0 ¯1 0││
│ │ │ │├────────┼────────┼────────┤│
│ │ │ ││0 0 0 ¯1│¯1 0 0 0│0 1 0 0 ││
│ │ │ │├────────┼────────┼────────┤│
│ │ │ ││0 0 1 0 │0 ¯1 0 0│¯1 0 0 0││
│ │ │ │└────────┴────────┴────────┘│
└───────┴───────┴───────┴────────────────────────────┘

qg ∆Q qg → 1.4E¯4 | 0% 
 qg ∆Q3 qg → 2.7E¯4 | +88% 
* qh ×H qh → 2.3E¯2 | +16405% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

qg ∘.×_∆Q qg → 2.3E¯5 | 0% 
 qg ∘.×_∆Q3 qg → 1.6E¯4 | +600% 
* qh ∘.(×H) qh → 2.8E¯2 | +120566% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌─────────────┬──────┐
│qg ∘.×_∆Q qg│18868 │
├─────────────┼──────┤
│qg ∘.×_∆Q3 qg│21484 │
├─────────────┼──────┤
│qh ∘.(×H) qh│120460│
└─────────────┴──────┘

Not bad!

## Projective geometric algebra

In [88]:
V ← {⍺←2*⍳≢⍵ ⋄ v←0⍴⍨1+⌈/,⍺ ⋄ v[⍺]+←(⍴⍺)↑⍵ ⋄ v}          
B ← {b←Z({⊂⊂⍵}⌸+/¨BB≢⍵)⌷¨⊂,⍵ ⋄ ⍺←⍳≢b ⋄ (⊂⍺)⌷(1+⌈/,⍺)X b}
G ← {⌈/+/¨(0≢¨C ⍵)/BB≢⍵}                                
D ← ⌈2⍟⌈⍥≢ 

# Summary

In [101]:
]defs 'M' 'U' 'I' 'BV' 'AV' 'VA'                      ⍝ vectors
]defs 'C' '_Z' 'X' '_X' 'S' 'Z' '_V'                  ⍝ vector extension
]defs 'V' 'B' 'G' 'D' 'R'                             ⍝ multivectors
]defs 'BB' 'PB' 'QS' '_SQ' 'BS' 'SB' 'TS' 'TX' 'TR'   ⍝ product table
]defs '_∆_' '_∆I' '_I' '_C' '_⍙_'                     ⍝ geometric operators
⍝]defs '∆' '∆X' '∆I' '∆∆' 'R' 'LC' 'RC' 'DC' '⍙' '⍙X' '⍙I'   ⍝ geometric functions

M ← 0.5*⍨+.×⍨ 
 U ← ÷∘(0.5*⍨+.×⍨)⍨ 
 I ← ÷∘(+.×⍨)⍨ 
BV ← {⍬≡⍵:0 ⋄ (1<≢⍵)∨(0<≡⍵)∧1<≢⊃⍵:⍵ ⋄ ⊃⍵}¨1↑⍨¨∘-1+⍳
AV ← ⊂⍤1↑[¯0.5] 
VA ← ⊂⍤¯1↑[¯0.5]

C ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ 1=≢∪⍵:⊃⍵ ⋄ ⍺∧.>|∊⍵:0 ⋄ ⍵}¨
_Z ← {⍺←⊢ ⋄ 0∊⍺ ⍵:0 ⋄ ⍺(⍺⍺)⍵} 
 X ← {⍺↑⍵,0⍴⍨0⌈⍺-≢⍵} 
_X ← {⍺←⊢ ⋄ ⍺(⍺⍺⍥((⍺(⍵⍵⍨⍨)⍵)∘X))⍵}(⌈⍥≢) 
 S ← {⍬≡⍵:0 ⋄ (1<≢⍵)∨(0<≡⍵)∧1<≢⊃⍵:⍵ ⋄ ⊃⍵} 
 Z ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ S ⍵↓⍨-⊥⍨(1∧.=1+∊)¨⍵} 
_V ← {⍺←⊢ ⋄ Z∘C ⍺(⍺⍺ _X)⍵}

V ← {⍺←2*⍳≢⍵ ⋄ v←0⍴⍨1+⌈/,⍺ ⋄ v[⍺]+←(⍴⍺)↑⍵ ⋄ v} 
B ← {b←Z({⊂⊂⍵}⌸+/¨BB≢⍵)⌷¨⊂,⍵ ⋄ ⍺←⍳≢b ⋄ (⊂⍺)⌷(1+⌈/,⍺)X b}
G ← {⌈/+/¨(0≢¨C ⍵)/BB≢⍵} 
D ← ⌈2⍟⌈⍥≢ 
R ← ×∘(¯1*(+\¯1↓⊢)+.×1↓⊣⍨¨↓∘⍉2⊥⍣¯1⍳∘≢)⍨

BB ← ↓∘⍉2⊥⍣¯1⍳ 
 PB ← 2⊥≠ 
 QS ← ⊃1 ¯1 0,.⍴⍨⊢↑⍨3⌈≢ 
_SQ ← {(⍺ SB ⍵)×(⌽⍺⍺)×.*⍺∧⍵} 
 BS ← (↓∘⍉2⊥⍣¯1⍳)2*+/ 
 SB ← ¯1*(+\¯1↓⊢)+.×1↓⊣ 
 TS ← {(∘.PB,⍥⊂∘.((QS ⍵)_SQ))⍨BS ⍵} 
 TX ← {(∘.PB,⍥⊂∘.((QS ⍵)_SQ))⍨BS ⍵}2⍴0,+/

_∆_ ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ r←(≢¨⍺ ⍵)⌊⍴p ⋄ Z(r↑p)V(r↑s)×_Z¨r↑⍺∘.(⍺⍺ _Z)⍥,⍵} 
_∆I ← {+/⍺(⍺⍺×_X×_X)⍵} 
 _I ← {i←Z∘C⊂⍤¯1⍉⍤¯1⍉((1↑⍨≢)⌹⍤2(↑[¯0.5]⍣2⊂⍺⍺¨BV∘≢))(2*D ⍵)X ⍵ ⋄ 0=⎕NC'⍺':i ⋄ ⍺ ⍺⍺ i}
 _C ← {⍺←D ⍵ ⋄ (⍺⍺×_V⌽)⍵ X⍨2*⍺} 
_⍙_ ← {⍵⍵ LC ⍺ ⍺⍺⍥((⍺(⍵⍵⍨⍨)⍵)∘RC)⍵}

In [100]:
]dinput
GA ← {
    g ← ⎕NS⍬ ⋄ g.s g.b ← ⍵ (BV 2*d←+/⍵)
    g._Z ← _Z ⋄ g.V ← V ⋄ g.Z ← Z ⋄ g.S ← S ⋄ g.D ← D ⋄ g.BV ← BV ⋄ g.C ← C
    g._X ← _X ⋄ g.X ← X{⍺←≢b ⋄ ⍺(⍺⍺)⍵}
    
    g._∆ ← _∆_(TS⍵)   ⋄  g._∆X ← _∆_(TX⍵)    ⋄  g.R   ← ×∘TR⍨
    g. ∆ ← ×g._∆      ⋄  g. ∆V ← g.∆⍥V       ⋄  g.∆v  ← g.∆∘V          
    g.∆X ← ×g._∆X     ⋄  g.∆XV ← g.∆X⍥V      ⋄  g.∆Xv ← g.∆X∘V
    g.∆I ← g.s _∆I    ⋄  g.∆IV ← g.∆I⍥V      ⋄  g.∆Iv ← g.∆I∘V
    g.∆∆ ← g.∆ _I     ⋄  g.∆∆V ← g.∆∆⍥V      ⋄  g.∆∆v ← g.∆∆∘V
    g.LC ← g.∆ _C     ⋄  g. RC ← g.∆⍨_C      ⋄  g. DC ← g.LC⍣2
    g. ⍙ ← g.∆ _⍙_ d  ⋄  g. ⍙I ← g.∆I _⍙_ d  ⋄  g. ⍙X ← g.∆X _⍙_ d
    g
}

In [87]:
Assert a b ((GA 3    ).∆ _GA_ 3    ) u v w
Assert a b ((GA 0 2  ).∆ _GA_ 0 2  ) u v w
Assert a b ((GA 3 0 1).∆ _GA_ 3 0 1) u v w